# Imports

## Install Required Packages if needed

In [1]:
#!pip install dataretrieval

## Import Libraries

In [98]:
import pandas as pd #for creating dataframe
import pyodbc #working with ODBC databases
import numpy as np # for locating values in dataframes
from datetime import datetime, timedelta # for obtaining today's date and converting time
import os #for working with directories
import dataretrieval.nwis as nwis # retrieving water level data

## Manage directories

In [99]:
# my default directory is c:\Users\bmilinic\OneDrive - DOI\Documents\Python\bemidji
defaultdirectory = os.getcwd()
print(defaultdirectory)
# connect to the shared drive which holds the databases and files (must be connected to VPN)
os.chdir('P:/0083/analysis/DataCompilation/DataCompilationPy/create_master_oil_levels')
print(os.getcwd())

P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels
P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels


FUNCTIONS

In [100]:
def time_check(df, site_col_name, site_num='473413095053701'):
    display(df.loc[df[site_col_name]==site_num].tail(2))

## Import data using separate blocks of code
- for some reason, trying to connect to all the data at once takes like 4x the time and sometimes fails

In [101]:
# frontend db
access_dbfe_dir = r'P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_fe.accdb' # test dir
access_dbfe_dir = r'P:\0083\analysis\DataCompilation\BemidjiMasterSiteData_fe.accdb' # updated dir

# backend db
access_dbbe_dir = r'P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_be.accdb' # test dir
access_dbbe_dir = r'P:\0083\analysis\DataCompilation\BackendData\BemidjiMasterSiteData_be.accdb' # updated dir

In [102]:
#print('Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_fe.accdb;')
#print(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_fe.accdb;')

#access_dbfe_dir = r'P:\0083\analysis\DataCompilation\DataCompilationPy\local_access_db\BemidjiMasterSiteData_fe.accdb'
#print(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+ access_dbfe_dir + ';')

In [103]:
# FRONT END  
Gfe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+ access_dbfe_dir + ';')
c_fe = Gfe_db.cursor()
# BACK END
Gbe_db = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+ access_dbbe_dir + ';')
c_be = Gbe_db.cursor()

In [104]:
# df_sites and site list
c_fe.execute('select USGS_siteno from tblSites')
df = pd.read_sql('select USGS_siteno from tblSites', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\4021136965.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('select USGS_siteno from tblSites', Gfe_db)


In [105]:
# Drop none values, create a list out of the column, and create a string out of the column
df_sites = df.dropna()
list_sites = df_sites['USGS_siteno'].tolist()

In [106]:
tblOilLevels = pd.read_sql('select * from tblOilLevels', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\1409885058.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblOilLevels = pd.read_sql('select * from tblOilLevels', Gfe_db)


In [107]:
tblSites = pd.read_sql('select * from tblSites', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\3222357457.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblSites = pd.read_sql('select * from tblSites', Gfe_db)


In [108]:
tblWells = pd.read_sql('select * from tblWells', Gfe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\3479789449.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblWells = pd.read_sql('select * from tblWells', Gfe_db)


In [109]:
#tblcd_WaterLevelStatusCode = pd.read_sql("select * from tblcd_WaterLevelStatusCode", Gfe_db)  
tblcd_FluidLevelStatusCode = pd.read_sql("select * from tblcd_FluidLevelStatusCode", Gbe_db)
tblcd_FluidLevelMethodCode = pd.read_sql("select * from tblcd_FluidLevelMethodCode", Gbe_db) 

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\654939022.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblcd_FluidLevelStatusCode = pd.read_sql("select * from tblcd_FluidLevelStatusCode", Gbe_db)
C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\654939022.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tblcd_FluidLevelMethodCode = pd.read_sql("select * from tblcd_FluidLevelMethodCode", Gbe_db)


In [110]:
# oil tape corrections
tbl_OilTapeCorrections = pd.read_sql('select * from tbl_OilTapeCorrections', Gbe_db)

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\4260948905.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tbl_OilTapeCorrections = pd.read_sql('select * from tbl_OilTapeCorrections', Gbe_db)


In [111]:
len(list_sites)

614

In [112]:
tblcd_FluidLevelStatusCode

,WebQualifier_GWSIStatusCode,ShortName_AQTSReadingQualifier,FullName_AQTSReadingCommentText
0,O,NoMeasurement,Obstruction
1,W,NoMeasurement,Destroyed
2,P,Above or NoMeasurement,Active Pumping
3,L,Saline,None
4,V,ForeignSubstance,None
5,A,None,Atmospheric Pressure
6,I,None,Injection At Well
7,J,None,Injection Nearby
8,K,None,Cascading Water
9,M,None,Aquifer Contact Lost


In [113]:
# From nwis
data, mdata = nwis.get_gwlevels(sites=list_sites[0:400], datetime_index=False) # creates a tuple of dataframe and associated metadata
# needs to be chunked
dfwl_list = []
dfwl_list.append(data)

data, mdata = nwis.get_gwlevels(sites=list_sites[400:], datetime_index=False) 
dfwl_list.append(data)

dfwl = pd.concat(dfwl_list, ignore_index=True)

# don't need parameter_cd 72019 which is depth to water. Only need 626211 which is groundwater level above NAVD 
dfwl = dfwl.loc[dfwl['parameter_cd'] == '72019']
dfwl

c:\Users\bmilinic\AppData\Local\miniforge3\envs\bem_env2\Lib\site-packages\dataretrieval\nwis.py:1489: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,parameter_cd
1,USGS,473356095043701,ST,2019-06-28,14:56,UTC,0.00,NaN,NaN,NaN,USGS,m,1,S,D,A,72019
3,USGS,473356095043701,ST,2020-08-06,18:00,UTC,-1.60,NaN,NaN,NaN,USGS,m,1,S,D,A,72019
5,USGS,473356095043701,ST,2021-08-19,16:00,UTC,-0.16,NaN,NaN,NaN,USGS,m,0,S,D,A,72019
7,USGS,473356095043701,ST,2022-08-25,14:08,UTC,-1.06,NaN,NaN,NaN,USGS,m,1,S,D,A,72019
9,USGS,473356095043701,ST,2023-06-22,15:30,UTC,-1.18,NaN,NaN,NaN,USGS,m,0,S,D,A,72019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76364,USGS,473430095052201,GW,2020-08-06,19:00,UTC,23.33,NaN,NaN,1,USGS,m,2,S,V,A,72019
76367,USGS,473430095052201,GW,2021-08-19,22:05,UTC,NaN,NaN,NaN,6,USGS,m,2,S,V,A,72019
76370,USGS,473430095052201,GW,2022-08-26,19:36,UTC,23.01,NaN,NaN,1,USGS,m,2,S,V,A,72019
76373,USGS,473430095052201,GW,2023-06-22,20:36,UTC,23.73,NaN,NaN,1,USGS,m,2,S,V,A,72019


In [114]:
#From gwsi bmj
bmj3 = pd.read_csv(r'../create_site_info_files/data_inputs/gwsi_current/bmj3_fromPy.csv')
bmjmp = pd.read_csv(r'../create_site_info_files/data_inputs/gwsi_old/bmj_mpnt_fromPy.csv') # does not need updated...only Aquarious MP are updated I believe
bmjrmk = pd.read_csv(r'../create_site_info_files/data_inputs/gwsi_current/bmj_rmk_fromPy.csv') 

## Inspect Some data 

In [115]:
# close the cursors and connections to the databases 
c_fe.close()
Gfe_db.close()
c_be.close()
Gbe_db.close()

In [116]:
# Inspect dfwl
display(dfwl.loc[dfwl['lev_status_cd'] == 'P'].head(2))
display(dfwl.lev_status_cd.value_counts())
dfwl.to_csv("outputs/tests/{}_test_NWIS_output.csv".format(str(datetime.now().date())), index=False)
print('Copy of NWIS output saved to', os.getcwd())

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,parameter_cd
10588,USGS,473425095052010,GW,2014-07-25,20:13,UTC,26.41,NaN,NaN,P,USGS,m,2,S,V,A,72019
11578,USGS,473425095052012,GW,2014-07-25,20:43,UTC,26.10,NaN,NaN,P,USGS,m,2,S,V,A,72019


lev_status_cd
1    24725
8      378
D      322
P       12
6        4
F        1
Name: count, dtype: int64

Copy of NWIS output saved to P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels


In [117]:
x = pd.read_csv(r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels' + '/outputs/tests/2024-10-24_test_NWIS_output.csv')
x.head(2)

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,parameter_cd
0,USGS,473356095043701,ST,2019-06-28,14:56,UTC,NaN,1390.17,NAVD88,NaN,USGS,m,1,S,D,A,62611
1,USGS,473356095043701,ST,2019-06-28,14:56,UTC,0.0,NaN,NaN,NaN,USGS,m,1,S,D,A,72019


In [118]:
y = pd.read_csv(r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels' + '/outputs/tests/2024-07-19_test_NWIS_output.csv')
y.head(2)

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd
0,USGS,473356095043701,ST,2019-06-28,09:56,UTC,0.0,NaN,NaN,NaN,USGS,m,1,S,D,A
1,USGS,473356095043701,ST,2020-08-06,13:00,UTC,-1.6,NaN,NaN,NaN,USGS,m,1,S,D,A


In [119]:
# r line 34 : edit tblOilLevels. First inspect data
display(tblOilLevels.shape)

(20161, 32)

# bmj.mpnt modified to get date ranges for MP heights

In [120]:
# r line 148
bmjmp2 = bmjmp.copy()
# r line 152 - convert columns to datetimes
bmjmp2['GWSI_MP_BeginDate'] = pd.to_datetime(bmjmp2['GWSI_MP_BeginDate'], format = '%Y%m%d')
bmjmp2['GWSI_MP_EndDate'] = pd.to_datetime(bmjmp2['GWSI_MP_EndDate'], format = '%Y%m%d')
# r line 160 - replace NaT EndDate values with today's date
today = pd.Timestamp.today().strftime('%Y-%m-%d')
bmjmp2['GWSI_MP_EndDate'] = bmjmp2['GWSI_MP_EndDate'].fillna(today)
# convert data type to make it mergable with tblOilLevels and 
# make bmjmp column mergeable 
bmjmp2['GWSI_USGS_siteno'] = bmjmp2['GWSI_USGS_siteno'].astype(str) 

In [121]:
# used to compare original df with new df
display(bmjmp.iloc[0:4, 0:4])
display(bmjmp2.iloc[0:4, 0:4])
bmjmp2.iloc[:, 0:4].dtypes

,GWSI_AgencyCode,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate
0,USGS,473429095051006,20160726.0,NaN
1,USGS,473424095052912,20160802.0,NaN
2,USGS,473424095052906,20160809.0,NaN
3,USGS,473423095052902,20180625.0,NaN


,GWSI_AgencyCode,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate
0,USGS,473429095051006,2016-07-26,2025-02-21
1,USGS,473424095052912,2016-08-02,2025-02-21
2,USGS,473424095052906,2016-08-09,2025-02-21
3,USGS,473423095052902,2018-06-25,2025-02-21


GWSI_AgencyCode              object
GWSI_USGS_siteno             object
GWSI_MP_BeginDate    datetime64[ns]
GWSI_MP_EndDate      datetime64[ns]
dtype: object

# Water level data from NWIS

In [122]:
# CHECK RANDOM ROWS

chk = dfwl.copy()
display(chk[['site_no', 'lev_tm', 'lev_tz_cd']].iloc[[0, 1, 7483, 7125]])
chk['lev_tm'] = chk['lev_tm'].fillna('18:01')
display(chk[['site_no', 'lev_tm', 'lev_tz_cd']].iloc[[0, 1, 7483, 7125]])

# new: correct level time to match oil data (and data release) 
chk['lev_tm'] = pd.to_datetime(chk['lev_tm'], format='%H:%M') # convert str to dt
chk['lev_tm'] = chk['lev_tm'] - timedelta(hours=6) # sub 6 hours to go from UTC to CST
chk['lev_tm'] = chk['lev_tm'].dt.strftime('%H:%M') # convert back to string
display(chk[['site_no', 'lev_tm', 'lev_tz_cd']].iloc[[0, 1, 7483, 7125]])


,site_no,lev_tm,lev_tz_cd
1,473356095043701,14:56,UTC
3,473356095043701,18:00,UTC
22417,473426095052526,NaN,NaN
21343,473426095052410,NaN,NaN


,site_no,lev_tm,lev_tz_cd
1,473356095043701,14:56,UTC
3,473356095043701,18:00,UTC
22417,473426095052526,18:01,NaN
21343,473426095052410,18:01,NaN


,site_no,lev_tm,lev_tz_cd
1,473356095043701,08:56,UTC
3,473356095043701,12:00,UTC
22417,473426095052526,12:01,NaN
21343,473426095052410,12:01,NaN


In [123]:
# CHECK specific site

chk = dfwl.copy()
display(chk[chk['site_no'] == '473413095053701'].tail(2))
chk['lev_tm'] = chk['lev_tm'].fillna('18:01')
display(chk[chk['site_no'] == '473413095053701'].tail(2))

# new: correct level time to match oil data (and data release) 
chk['lev_tm'] = pd.to_datetime(chk['lev_tm'], format='%H:%M') # convert str to dt
chk['lev_tm'] = chk['lev_tm'] - timedelta(hours=6) # sub 6 hours to go from UTC to CST
chk['lev_tm'] = chk['lev_tm'].dt.strftime('%H:%M') # convert back to string
display(chk[chk['site_no'] == '473413095053701'].tail(2))

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,parameter_cd
380,USGS,473413095053701,GW,2023-06-21,21:32,UTC,18.02,NaN,NaN,1,USGS,m,2,S,V,A,72019
383,USGS,473413095053701,GW,2024-07-25,16:45,UTC,17.73,NaN,NaN,1,USGS,m,2,S,V,A,72019


,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,parameter_cd
380,USGS,473413095053701,GW,2023-06-21,21:32,UTC,18.02,NaN,NaN,1,USGS,m,2,S,V,A,72019
383,USGS,473413095053701,GW,2024-07-25,16:45,UTC,17.73,NaN,NaN,1,USGS,m,2,S,V,A,72019


,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,parameter_cd
380,USGS,473413095053701,GW,2023-06-21,15:32,UTC,18.02,NaN,NaN,1,USGS,m,2,S,V,A,72019
383,USGS,473413095053701,GW,2024-07-25,10:45,UTC,17.73,NaN,NaN,1,USGS,m,2,S,V,A,72019


In [124]:
# repetative but safe
# TODO fix - probably erase
# dfwl = data[0].copy()

# inspect nan and other data before change
display(dfwl['lev_tm'].iloc[[0, 1, 7483, 7125]])

# r line 202
# replace blank times with 1701 which will be converted to 1201 pm or 1201 (matches field parameter approach)
dfwl['lev_tm'] = dfwl['lev_tm'].fillna('18:01')

# new: correct level time to match oil data (and data release) 
dfwl['lev_tm'] = pd.to_datetime(dfwl['lev_tm'], format='%H:%M') # convert str to dt
dfwl['lev_tm'] = dfwl['lev_tm'] - timedelta(hours=6) # sub 6 hours to go from UTC to CST
dfwl['lev_tm'] = dfwl['lev_tm'].dt.strftime('%H:%M') # convert back to string

1        14:56
3        18:00
22417      NaN
21343      NaN
Name: lev_tm, dtype: object

In [125]:
time_check(dfwl, 'site_no')

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,lev_agency_cd,lev_dt_acy_cd,lev_acy_cd,lev_src_cd,lev_meth_cd,lev_age_cd,parameter_cd
380,USGS,473413095053701,GW,2023-06-21,15:32,UTC,18.02,NaN,NaN,1,USGS,m,2,S,V,A,72019
383,USGS,473413095053701,GW,2024-07-25,10:45,UTC,17.73,NaN,NaN,1,USGS,m,2,S,V,A,72019


In [126]:
# inspect data after change
display(dfwl['lev_tm'].iloc[[0, 1, 7483, 7125]])

1        08:56
3        12:00
22417    12:01
21343    12:01
Name: lev_tm, dtype: object

In [127]:
# check for merge
print(dfwl['site_no'].dtype)
print(bmjmp2.GWSI_USGS_siteno.dtype)

object
object


In [128]:
tblcd_FluidLevelStatusCode

,WebQualifier_GWSIStatusCode,ShortName_AQTSReadingQualifier,FullName_AQTSReadingCommentText
0,O,NoMeasurement,Obstruction
1,W,NoMeasurement,Destroyed
2,P,Above or NoMeasurement,Active Pumping
3,L,Saline,None
4,V,ForeignSubstance,None
5,A,None,Atmospheric Pressure
6,I,None,Injection At Well
7,J,None,Injection Nearby
8,K,None,Cascading Water
9,M,None,Aquifer Contact Lost


In [129]:
# check for merge
print(dfwl['lev_status_cd'], type(dfwl['lev_status_cd']))
print(tblcd_FluidLevelStatusCode['WebQualifier_GWSIStatusCode'])

1        NaN
3        NaN
5        NaN
7        NaN
9        NaN
        ... 
76364      1
76367      6
76370      1
76373      1
76376      1
Name: lev_status_cd, Length: 25491, dtype: object <class 'pandas.core.series.Series'>
0     O
1     W
2     P
3     L
4     V
5     A
6     I
7     J
8     K
9     M
10    N
11    Z
12    B
13    1
14    2
15    3
16    4
17    5
18    6
19    7
20    8
21    9
22    D
23    E
24    G
25    H
26    R
27    S
28    T
29    X
30    C
31    F
Name: WebQualifier_GWSIStatusCode, dtype: object


In [130]:
dfwl2 = dfwl.copy()

# r line 210, merge dfs to attach descriptors
dfwl2 = pd.merge(dfwl2, tblcd_FluidLevelStatusCode.loc[:, ['WebQualifier_GWSIStatusCode', 'FullName_AQTSReadingCommentText']], how='left', left_on='lev_status_cd', right_on='WebQualifier_GWSIStatusCode')

# Convert column to make the NANs mergable
#dfwl2.lev_status_cd = dfwl2.lev_status_cd.astype('int64') # issue...can't convert NaN values...need new method
dfwl2.lev_status_cd = pd.to_numeric(dfwl2.lev_status_cd, errors='coerce')

# Back to merging
#dfwl2 = pd.merge(dfwl2, tblcd_WaterLevelStatusCode.loc[:, ['lev_status_cd_ID', 'Comments_WaterLevelStatus']], how='left', left_on= 'lev_status_cd', right_on= 'lev_status_cd_ID')
dfwl2 = pd.merge(dfwl2, tblcd_FluidLevelMethodCode.loc[:, ['lev_meth_cd', 'Comments_FluidLevelMethod']], how='left', on= 'lev_meth_cd')
print(dfwl2.shape)

# r line 220 merge with mpnt table to get current mp height
dfwl2['site_no'] = dfwl2['site_no'].astype(str)  # convert column to integer to make it mergible
dfwl2 = pd.merge(dfwl2, bmjmp2.loc[:, ["GWSI_USGS_siteno","GWSI_MP_BeginDate","GWSI_MP_EndDate","GWSI_MP_height_ft"]], how = 'left', left_on='site_no', right_on='GWSI_USGS_siteno')
print(dfwl2.shape)

# change the FullName col name
dfwl2.rename(columns={'FullName_AQTSReadingCommentText':'Comments_FluidLevelStatus'}, inplace=True)



(25491, 20)
(36305, 24)


In [131]:
# INSPECT DATA: lev_dt should be between begin and end dates
dfwl2.loc[:, ['GWSI_MP_BeginDate', 'lev_dt', 'GWSI_MP_EndDate']]

,GWSI_MP_BeginDate,lev_dt,GWSI_MP_EndDate
0,2019-06-28,2019-06-28,2025-02-21
1,2019-06-28,2020-08-06,2025-02-21
2,2019-06-28,2021-08-19,2025-02-21
3,2019-06-28,2022-08-25,2025-02-21
4,2019-06-28,2023-06-22,2025-02-21
...,...,...,...
36300,2010-03-10,2022-08-26,2025-02-21
36301,1990-07-02,2023-06-22,2010-03-09
36302,2010-03-10,2023-06-22,2025-02-21
36303,1990-07-02,2024-07-25,2010-03-09


In [132]:
# r script 227 
# INSPECT DATA: check to see where rows for MP height are not valid based on date
chk = dfwl2[
    ~(dfwl2['lev_dt'] >= dfwl2['GWSI_MP_BeginDate']) |
    ~(dfwl2['lev_dt'] <= dfwl2['GWSI_MP_EndDate'])
    ]

chk.loc[:, ['site_no', 'GWSI_MP_BeginDate', 'lev_dt', 'GWSI_MP_EndDate']]

print('The number of rows lost is...', chk.shape[0])

The number of rows lost is... 10875


Only select rows where the MP height is valid for the date the level was taken

In [133]:
# r script 227 too
# select only the rows where the MP height is valid for the date the level was taken (make sure this is after)
print(dfwl2.shape)
dfwl2 = dfwl2[
    (dfwl2['lev_dt'] >= dfwl2['GWSI_MP_BeginDate']) &
    (dfwl2['lev_dt'] <= dfwl2['GWSI_MP_EndDate'])
    ]
print(dfwl2.shape)

(36305, 24)
(25430, 24)


In [134]:
# INSPECT the updated data
dfwl2.loc[:, ['GWSI_MP_BeginDate', 'lev_dt', 'GWSI_MP_EndDate']]

,GWSI_MP_BeginDate,lev_dt,GWSI_MP_EndDate
0,2019-06-28,2019-06-28,2025-02-21
1,2019-06-28,2020-08-06,2025-02-21
2,2019-06-28,2021-08-19,2025-02-21
3,2019-06-28,2022-08-25,2025-02-21
4,2019-06-28,2023-06-22,2025-02-21
...,...,...,...
36296,2010-03-10,2020-08-06,2025-02-21
36298,2010-03-10,2021-08-19,2025-02-21
36300,2010-03-10,2022-08-26,2025-02-21
36302,2010-03-10,2023-06-22,2025-02-21


## Average water level creation

In [135]:
# r script 244
# create average water level by site data frame for determining water table wells
# Lev.avg used later
wl_avg = dfwl2.groupby('site_no')['lev_va'].mean().reset_index(name='mean_lev_va') # note, .reset_index gives the series an index and turns it into a data frame

In [136]:
# r line 250
# INSPECT: check for NaN
# this could be due to a well being dry or affected by ice
display(wl_avg[wl_avg.mean_lev_va.isna()])
# check these rows
dfwl2.loc[(dfwl2['site_no'] == '473416095052601')|
          (dfwl2['site_no'] == '473420095051601')|
          (dfwl2['site_no'] == '473420095052501')]

#Note what makes this .loc unique...based on individual values inside a cell rather than columns

,site_no,mean_lev_va
12,473416095052601,NaN


,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
19691,USGS,473416095052601,GW,1990-10-10,12:01,NaN,NaN,NaN,NaN,NaN,...,V,A,72019,D,Dry,Calibrated electric tape – accuracy of instrum...,473416095052601,1990-06-24,2025-02-21,3.13
19692,USGS,473416095052601,GW,1990-10-26,12:01,NaN,NaN,NaN,NaN,NaN,...,V,A,72019,D,Dry,Calibrated electric tape – accuracy of instrum...,473416095052601,1990-06-24,2025-02-21,3.13
19693,USGS,473416095052601,GW,1991-03-09,12:01,NaN,NaN,NaN,NaN,NaN,...,V,A,72019,D,Dry,Calibrated electric tape – accuracy of instrum...,473416095052601,1990-06-24,2025-02-21,3.13
19694,USGS,473416095052601,GW,1992-06-16,12:01,NaN,NaN,NaN,NaN,NaN,...,V,A,72019,D,Dry,Calibrated electric tape – accuracy of instrum...,473416095052601,1990-06-24,2025-02-21,3.13
20657,USGS,473420095051601,GW,1983-05-25,12:01,NaN,24.21,NaN,NaN,1.0,...,S,A,72019,1,Static,Steel-tape,473420095051601,1983-05-24,2010-03-08,3.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21390,USGS,473420095052501,GW,1994-12-08,12:01,NaN,5.13,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473420095052501,1989-10-26,2025-02-21,1.21
21392,USGS,473420095052501,GW,1995-01-03,12:01,NaN,5.52,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473420095052501,1989-10-26,2025-02-21,1.21
21394,USGS,473420095052501,GW,1995-03-17,12:01,NaN,5.08,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473420095052501,1989-10-26,2025-02-21,1.21
21396,USGS,473420095052501,GW,1995-04-28,12:01,NaN,4.94,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473420095052501,1989-10-26,2025-02-21,1.21


Checking negative mean levels and looking at their sites to determine if they are in a lake or near a wetlands so that negative levels are ok

In [137]:
# INSPECTION
# r line 255 checking negative mean levels
wl_neg = dfwl2.loc[dfwl2['lev_va'] < 0].drop_duplicates(subset = 'site_no')

# r line 260
tblSites.loc[tblSites['USGS_siteno'].isin(wl_neg['site_no']), 'LocalSiteName'] # returns only rows where condition is true and the LocalSiteName column

239               WL02
240               WL03
241               WL04
242     Grant Creek 01
243             WL01-2
244               LK01
997                313
1053               426
1055              501A
1056              501B
1057              501C
1196              9003
1204              9011
Name: LocalSiteName, dtype: object

Using bmj3 data

In [138]:
# r line 323. Determine if wells are water table wells (ie screened interval within 1 m of average water level)
# merge bmj and dfwl2
print(bmj3.GWSI_USGS_siteno.dtypes, wl_avg.site_no.dtypes) # appears the dtypes are dif 
bmj3.GWSI_USGS_siteno = bmj3.GWSI_USGS_siteno.astype(str)
print(bmj3.GWSI_USGS_siteno.dtypes, wl_avg.site_no.dtypes)
bmj3_2 = pd.merge(bmj3, wl_avg, how='left', left_on ='GWSI_USGS_siteno', right_on = 'site_no')

int64 object
object object


In [139]:
# INSPECT
# check how many rows don't have screen info
print(bmj3_2.loc[bmj3_2['GWSI_TopOfScreenDepth_ftBLS'].isna()].shape[0])
print(bmj3_2.loc[bmj3_2['GWSI_BottomOfScreenDepth_ftBLS'].isna()].shape[0])

118
118


In [140]:
# INSPECT (for Andrew)
# r 340 make a list of sites that didn't come out in a gwsi retrieval for Andrew to check:
# make boolean mask to indicate where tblSites is not na for two columns and merge that with bmj3_2
GWSI_not_retrieved = pd.merge(tblSites.loc[tblSites['USGS_siteno'].notna(), ['USGS_siteno', 'LocalSiteName']], bmj3_2, left_on='USGS_siteno', right_on='GWSI_USGS_siteno', how='outer', indicator=True)
# create df where only data in left exists (shows where data on right is missing)
GWSI_not_retrieved = GWSI_not_retrieved.loc[GWSI_not_retrieved['_merge'] == 'left_only', ['USGS_siteno', 'LocalSiteName']]
# drops old index and replaces it with a new default one (meaning 0,1,2...)
GWSI_not_retrieved = GWSI_not_retrieved.reset_index(drop=True)
today = str(datetime.now().date())
GWSI_not_retrieved.to_csv(f'DataChecks/{today}_GWSI_not_retrieved.csv', index = False) 
print('File location: ', os.getcwd()+r'\DataChecks')

File location:  P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\DataChecks


In [141]:
# create a new column and set it to NA
bmj3_2['WaterTableWell'] = pd.NA

# assign 'Y' to wells that meet the criteria. Select rows where criteria is met...and change WaterTableWell column to 'Y' if it does
# 3.2808... is the screened interval value in ft aka 1 m
bmj3_2.loc[(bmj3_2['mean_lev_va'] < bmj3_2['GWSI_BottomOfScreenDepth_ftBLS']) &   # check that mean elevation falls within the screened interval
            (bmj3_2['mean_lev_va'] > bmj3_2['GWSI_TopOfScreenDepth_ftBLS']) | 
            (bmj3_2['mean_lev_va'] > bmj3_2['GWSI_TopOfScreenDepth_ftBLS'] - 3.2808399) & # check that mean elevation is within 1 m above the top of screened interval
            (bmj3_2['mean_lev_va'] < bmj3_2['GWSI_TopOfScreenDepth_ftBLS']) | 
            (bmj3_2['mean_lev_va'] < bmj3_2['GWSI_BottomOfScreenDepth_ftBLS'] + 3.2808399) & # check taht mean elevation is within 1 m below the bottom of the screened interval
            (bmj3_2['mean_lev_va'] > bmj3_2['GWSI_BottomOfScreenDepth_ftBLS']), 
            'WaterTableWell'] = 'Y' # if any set of that conditional statements are true, na is set to 'Y'

In [142]:
# INSPECT
# r 353 check number of water table wells
# .size() checks the number of times a value appears and resetting the index just sets it back to zero and creates a column called count
bmj3_2.groupby('WaterTableWell').size().reset_index(name='count')

,WaterTableWell,count
0,Y,149


In [143]:
# INSPECT
# r 357 check the logic for assigning water table status
# 0 means logic works ok

# Filter for wells with "Y" status to check logic (should be zero)
filter1 = (bmj3_2["WaterTableWell"]=="Y") & (bmj3_2["GWSI_TopOfScreenDepth_ftBLS"]-bmj3_2["mean_lev_va"]>3.2808)
print(bmj3_2[filter1].shape[0])

# Assign "N" status where logic fails
filter2 = bmj3_2["GWSI_TopOfScreenDepth_ftBLS"]-bmj3_2["mean_lev_va"]>3.2808
bmj3_2.loc[filter2, "WaterTableWell"] = "N" # Assign 'N' to WaterTableWell if Filter is true

# Group and count number of wells with each status
chk = bmj3_2.groupby('WaterTableWell').agg('count')
chk[['GWSI_USGS_siteno', 'site_no']]


0


,GWSI_USGS_siteno,site_no
WaterTableWell,,
N,126,126
Y,149,149


In [144]:
# r 367 check other site types
print(bmj3_2['GWSI_GWSISiteType'].unique())
# number of rows in the data where column is equal to WE or LK or ST
print(bmj3_2[(bmj3_2['GWSI_GWSISiteType'] == 'WE') | 
             (bmj3_2['GWSI_GWSISiteType'] == 'LK') | 
             (bmj3_2['GWSI_GWSISiteType'] == 'ST')].shape[0]) 

#all of these site types are surface-water and so appropriate for 
#water table mapping, assign "Y" in water table well column
bmj3_2.loc[(bmj3_2['GWSI_GWSISiteType'] == 'WE') | 
           (bmj3_2['GWSI_GWSISiteType'] == 'LK') | 
           (bmj3_2['GWSI_GWSISiteType'] == 'ST'), # for rows that equal WE, LK, or ST
           'WaterTableWell'] = 'Y'                # and only the WaterTableWell column...set it equal to 'Y'

bmj3_2.groupby('WaterTableWell').size().reset_index(name='counts') # another way of creating this same output as before

['GW' 'SB-UZ' 'WE' 'ST' 'LK' 'GW-TH']
11


,WaterTableWell,counts
0,N,126
1,Y,160


In [145]:
# r 380 create new columns with mid values (new method)
bmj3_tmp = bmj3_2.copy()
bmj3_tmp['MidOfScreenDepth_ft'] = round(0.5*(bmj3_tmp['GWSI_TopOfScreenDepth_ftBLS']+bmj3_tmp['GWSI_BottomOfScreenDepth_ftBLS']), 2)
bmj3_tmp['MidOfScreenDepth_m'] = round(bmj3_tmp['MidOfScreenDepth_ft']*0.3048, 3)
bmj3_tmp['MidOfScreenElevation_ftASL_NAVD88'] = bmj3_tmp['GWSI_LandSurfaceAltitude_ftASL_NAVD88']-bmj3_tmp['MidOfScreenDepth_ft']
bmj3_tmp['MidOfScreenElevation_mASL_NAVD88'] = round(bmj3_tmp['MidOfScreenElevation_ftASL_NAVD88']*0.3048, 3)
bmj3_tmp = bmj3_tmp.loc[:,['GWSI_USGS_siteno', 
                           'GWSI_LandSurfaceAltitude_ftASL_NAVD88', 
                           'GWSI_LandSurfaceAltitude_mASL_NAVD88', 
                           'MidOfScreenDepth_ft', 
                           'MidOfScreenDepth_m', 
                           'MidOfScreenElevation_ftASL_NAVD88', 
                           'MidOfScreenElevation_mASL_NAVD88',
                           'WaterTableWell']]
print(bmj3_tmp.keys())

Index(['GWSI_USGS_siteno', 'GWSI_LandSurfaceAltitude_ftASL_NAVD88',
       'GWSI_LandSurfaceAltitude_mASL_NAVD88', 'MidOfScreenDepth_ft',
       'MidOfScreenDepth_m', 'MidOfScreenElevation_ftASL_NAVD88',
       'MidOfScreenElevation_mASL_NAVD88', 'WaterTableWell'],
      dtype='object')


Moving on to tbl_OilTapeCorrection

In [146]:
# INSPECT
# r line 73 : edit tbl_OilTapeCorrections (tape.core)
print(tbl_OilTapeCorrections.shape)
display(tbl_OilTapeCorrections.StartDate.sample(3))
print(tbl_OilTapeCorrections.EndDate)

(22, 11)


14   2021-11-24
15   2021-11-24
21   2024-07-09
Name: StartDate, dtype: datetime64[ns]

0    2016-11-17
1    2018-06-24
2    2018-06-24
3    2021-11-23
4    2021-11-23
5    2021-11-23
6    2021-11-23
7    2018-06-25
8    2019-10-19
9    2019-10-19
10   2024-07-08
11   2024-07-08
12   2024-07-08
13   2024-07-08
14   2024-07-08
15   2024-07-08
16   2024-07-08
17          NaT
18          NaT
19          NaT
20          NaT
21          NaT
Name: EndDate, dtype: datetime64[ns]


In [147]:
tbl_OilTapeCorrections2 = tbl_OilTapeCorrections.copy()
tbl_OilTapeCorrections2['EndDate'] = tbl_OilTapeCorrections2['EndDate'].fillna(pd.Timestamp.today()) # replace unknown end dates with today's date. 
tbl_OilTapeCorrections2['EndDate'] = tbl_OilTapeCorrections2['EndDate'].dt.date # changes datetime obj to string (I think) to remove the hr, min, sec 
tbl_OilTapeCorrections2.EndDate

0     2016-11-17
1     2018-06-24
2     2018-06-24
3     2021-11-23
4     2021-11-23
5     2021-11-23
6     2021-11-23
7     2018-06-25
8     2019-10-19
9     2019-10-19
10    2024-07-08
11    2024-07-08
12    2024-07-08
13    2024-07-08
14    2024-07-08
15    2024-07-08
16    2024-07-08
17    2025-02-21
18    2025-02-21
19    2025-02-21
20    2025-02-21
21    2025-02-21
Name: EndDate, dtype: object

# tbloillevels

In [148]:
tblOilLevels.OilLevelTime

0        17:01
1        16:25
2        16:58
3        16:47
4        16:22
         ...  
20156    12:01
20157    12:01
20158    12:01
20159    12:01
20160    12:01
Name: OilLevelTime, Length: 20161, dtype: object

In [149]:
tblOilLevels2 = tblOilLevels.copy()

# correct time if in CDT timezone
tblOilLevels2['OilLevelTime'] = pd.to_datetime(tblOilLevels2['OilLevelTime']) # convert str to datetime
tblOilLevels2.loc[tblOilLevels2['TimeDatum']=='CDT', 'OilLevelTime'] = tblOilLevels2.loc[tblOilLevels2['TimeDatum']=='CDT', 'OilLevelTime'] - pd.Timedelta(hours=1) # subtract 1
tblOilLevels2['OilLevelTime'] = tblOilLevels2['OilLevelTime'].dt.strftime('%H:%M') # convert datetime back to str

# convert date and edit cols/rows
tblOilLevels2['OilLevelDate'] = pd.to_datetime(tblOilLevels['OilLevelDate'], format = '%Y-%m-%d') # make sure this col is not datatime
tblOilLevels2 = tblOilLevels2[tblOilLevels2['ExcludeFromPublishing'] != 1] # only keep rows that are not excluded from publishing
tblOilLevels2 = tblOilLevels2.loc[:, ~tblOilLevels2.columns.str.startswith('Excl')]
tblOilLevels2 = tblOilLevels2.loc[:, ~tblOilLevels2.columns.str.startswith('ARCHIVE')]
tblOilLevels2 = tblOilLevels2.loc[:, ~tblOilLevels2.columns.str.startswith('DELETE')]

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\1942304333.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  tblOilLevels2['OilLevelTime'] = pd.to_datetime(tblOilLevels2['OilLevelTime']) # convert str to datetime


In [150]:
# Inspect DELETE
# No rounding issue
display(tblOilLevels[['LocalSiteName','OilLevelTime', 'OilLevel_ftBMP']].loc[tblOilLevels['LocalSiteName']=='0501'])

,LocalSiteName,OilLevelTime,OilLevel_ftBMP
50,0501,14:30,22.72
152,0501,14:44,21.50
271,0501,17:04,22.71
19534,0501,12:01,NaN
19735,0501,14:46,22.55
19906,0501,11:38,22.41
19973,0501,11:14,21.78
20063,0501,16:09,22.83
20131,0501,10:04,23.06


In [151]:
# INSPECT
print("Old number of rows then columns:", tblOilLevels.shape, "New number of rows then columns:", tblOilLevels2.shape)
# check if all rows have a valid date value
display(tblOilLevels2.loc[tblOilLevels2['OilLevelDate'].isna(), :])
# check if all rows have a valid date value
display(tblOilLevels2.loc[tblOilLevels2['OilLevelTime'].isna(), :])

Old number of rows then columns: (20161, 32) New number of rows then columns: (19689, 12)


,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,OilLevelDataSource,ZeroThicknessCalcExclude
2736,20272,None,None,NaT,NaN,None,0.0,None,None,None,None,None


,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,OilLevelDataSource,ZeroThicknessCalcExclude
2736,20272,None,None,NaT,NaN,None,0.0,None,None,None,None,None
2782,20318,9017,None,2021-08-11,NaN,None,0.0,None,None,None,None,None


In [152]:
# INSPECT
# Compare to tblsites
display(tblOilLevels2.LocalSiteName)
display(tblSites.USGS_siteno)

0         315
1         317
2         319
3         411
4        420A
         ... 
20156     319
20157     411
20158    421B
20159    301A
20160     315
Name: LocalSiteName, Length: 19689, dtype: object

0                  None
1                  None
2                  None
3                  None
4                  None
             ...       
1796               None
1797    473424095053001
1798               None
1799               None
1800               None
Name: USGS_siteno, Length: 1801, dtype: object

In [153]:
#merge tblOilLevels with tblSites to get USGS_siteno r line 80
tblOilLevels3 = pd.merge(tblOilLevels2, tblSites.loc[:,['LocalSiteName', 'USGS_siteno',"AgencyCode","XcoordUTMNAD83_m","YcoordUTMNAD83_m","OnNWT","NWTPosition_m"]], how = 'left')
print("Old number rows/columns:", tblOilLevels2.shape, "New number rows/columns:", tblOilLevels3.shape)
tblOilLevels3 = pd.merge(tblOilLevels3, tblWells.loc[:,['LocalSiteName', 'LocalUseCode']], how = 'left')
print("Newest number rows/columns:", tblOilLevels3.shape)

Old number rows/columns: (19689, 12) New number rows/columns: (19689, 18)
Newest number rows/columns: (19689, 19)


In [154]:
# INSPECT
# check which records don't have USGS_siteno
display(tblOilLevels3.loc[tblOilLevels3['USGS_siteno'].isna(), ['LocalSiteName', 'USGS_siteno']].head(2))
display(tblOilLevels3.loc[tblOilLevels3['USGS_siteno'].notna(), ['LocalSiteName', 'USGS_siteno']].head(2))

# check which records don't have LocaluseCode
display(tblOilLevels3.loc[tblOilLevels3['LocalUseCode'].isna(), ['LocalSiteName', 'LocalUseCode']].head(2))
display(tblOilLevels3.loc[tblOilLevels3['LocalUseCode'].notna(), ['LocalSiteName', 'LocalUseCode']].head(2))

,LocalSiteName,USGS_siteno
2732,None,NaN


,LocalSiteName,USGS_siteno
0,315,473426095052530
1,317,473425095052009


,LocalSiteName,LocalUseCode
2732,None,NaN
4433,LG,NaN


,LocalSiteName,LocalUseCode
0,315,wo
1,317,wo


In [155]:
# INSPECT
print("Old number of rows then columns:", tblOilLevels2.shape, "New number of rows then columns:", tblOilLevels3.shape)

Old number of rows then columns: (19689, 12) New number of rows then columns: (19689, 19)


DUH merge ONLY localSiteName and USGS_siteno from tblsites rather than merge ON those columns from the two dfs.

In [156]:
# r line 93 select only the oil wells (LocalUseCode=="wo") and wells with a USGS site ID
tblOilLevels4 = tblOilLevels3[(tblOilLevels3['USGS_siteno'].notna()) & (tblOilLevels3['LocalUseCode']=='wo')]
print(tblOilLevels4.shape)

#rearrange order of columns, move last 2 columns from the join to the front
cols = tblOilLevels4.columns.tolist()      # create list of column names
new_cols = cols[-2:] + cols[:-2]           # change order by moving last 2 to the front
tblOilLevels4 = tblOilLevels4[new_cols]    # apply change to dataframe (create new one technically)

(4879, 19)


## Apply Tape Corrections to OilLevel_ftBMP

In [157]:
tblOilLevels5 = pd.merge(tblOilLevels4, tbl_OilTapeCorrections2, how = 'left', on = 'Oil_Tape_Serial_No')
tblOilLevels5.keys()

Index(['NWTPosition_m', 'LocalUseCode', 'OilLevelID', 'LocalSiteName',
       'PersonMakingMeasurement', 'OilLevelDate', 'OilLevelTime', 'TimeDatum',
       'OilLevel_ftBMP', 'OilLevelMethod', 'Oil_Tape_Serial_No',
       'Comments_OilLevel', 'OilLevelDataSource', 'ZeroThicknessCalcExclude',
       'USGS_siteno', 'AgencyCode', 'XcoordUTMNAD83_m', 'YcoordUTMNAD83_m',
       'OnNWT', 'TapeCorrID', 'HIF_ID', 'StartDate', 'EndDate',
       'StartDistance_ft', 'EndDistance_ft', 'GWSI_Tape_Correction_ft',
       'OilTapeModelMakeType', 'Source', 'Comments_TapeCorrection'],
      dtype='object')

In [158]:
# r line 127
tblOilLevels6 = tblOilLevels5[
    (tblOilLevels5['Oil_Tape_Serial_No'].isna()) | 
    (tblOilLevels5['OilLevel_ftBMP'].isna()) | 
    (tblOilLevels5['OilLevelDate'] >= tblOilLevels5['StartDate']) &
    (tblOilLevels5['OilLevelDate'] <= tblOilLevels5['EndDate']) &
    (tblOilLevels5['OilLevel_ftBMP'] >= tblOilLevels5['StartDistance_ft']) &
    (tblOilLevels5['OilLevel_ftBMP'] <= tblOilLevels5['EndDistance_ft'])
    ].drop_duplicates(subset=['OilLevelID'], keep = 'first')
# r line 131 
tblOilLevels6 = tblOilLevels6.drop(columns=[
    'TapeCorrID',
    'StartDate',
    'EndDate',
    'StartDistance_ft',
    'EndDistance_ft',
    'OilTapeModelMakeType',
    'Source',
    'Comments_TapeCorrection'    
])
# r line 132
tblOilLevels6['GWSI_Tape_Correction_ft'] = tblOilLevels6['GWSI_Tape_Correction_ft'].fillna(0)
# r line 133
tblOilLevels6['OilLevel_ftBMP_corr'] = tblOilLevels6['OilLevel_ftBMP'] + tblOilLevels6['GWSI_Tape_Correction_ft']

## Join oillevels with mpnt table

In [159]:
# r line 163 join OilLevels with MPNT table
# join
tblOilLevels7 = pd.merge(tblOilLevels6, bmjmp2.loc[:, ['GWSI_USGS_siteno', 'GWSI_MP_BeginDate', 'GWSI_MP_EndDate', 'GWSI_MP_height_ft']], how='left', left_on='USGS_siteno', right_on='GWSI_USGS_siteno')
# filter out rows where MP height is invalid for the date the level was taken
tblOilLevels7 = tblOilLevels7[
    (tblOilLevels7['OilLevelDate'] >= tblOilLevels7['GWSI_MP_BeginDate']) &
    (tblOilLevels7['OilLevelDate'] <= tblOilLevels7['GWSI_MP_EndDate'])
    ]

In [160]:
# INSPECT
# check row count
print("Number of rows before:", tblOilLevels6.shape[0])
print("Number of rows after:",tblOilLevels7.shape[0])
# another check that is similar
chk = tblOilLevels6.merge(tblOilLevels7, on='OilLevelID', how='outer', indicator=True) # merge all data with indicator that should if merge included both or some only 1 dataset
chk1 = chk[chk['_merge'] == 'left_only']
chk2 = chk[chk['_merge'] == 'right_only']  
print('Data from left only:', chk1.shape[0], 'Data from right only:', chk2.shape[0])

Number of rows before: 4879
Number of rows after: 4879
Data from left only: 0 Data from right only: 0


## Insert MP heights into tblOilLevels and calc oil level

In [161]:
# r code 180
tblOilLevels8 = tblOilLevels7.copy()
# calculate oil level
tblOilLevels8['OilLevel_ftBLS'] = tblOilLevels8['OilLevel_ftBMP_corr'] - tblOilLevels8['GWSI_MP_height_ft']
# r line 183 merge fluid level method
tblOilLevels8 = tblOilLevels8.merge(tblcd_FluidLevelMethodCode.loc[:, ['lev_meth_cd', 'Comments_FluidLevelMethod']], left_on='OilLevelMethod', right_on='lev_meth_cd', how='left') # forgot the how = left
# needed for the merge around r line 425
tblOilLevels8['OilLevelDate'] = tblOilLevels8['OilLevelDate'].dt.strftime('%Y-%m-%d')

# round col (apparently pd.dfs automatically round for you so you don't see it)
display(tblOilLevels8.loc[49:50, 'OilLevel_ftBLS'])
display(tblOilLevels8.loc[50, 'OilLevel_ftBLS'])
tblOilLevels8['OilLevel_ftBLS'] = round(tblOilLevels8['OilLevel_ftBLS'], 2)
display(tblOilLevels8.loc[50, 'OilLevel_ftBLS'])

49      NaN
50    20.19
Name: OilLevel_ftBLS, dtype: float64

np.float64(20.189999999999998)

np.float64(20.19)

# Create master list

In [162]:
time_check(dfwl2, 'site_no')

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
165,USGS,473413095053701,GW,2023-06-21,15:32,UTC,18.02,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473413095053701,1985-05-16,2025-02-21,2.56
167,USGS,473413095053701,GW,2024-07-25,10:45,UTC,17.73,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473413095053701,1985-05-16,2025-02-21,2.56


In [163]:
# INSPECTION
# r line 263 
#check each data set for duplicated values in USGS station ID,
#sample date and sample time columns.  
chk = tblOilLevels8.groupby(['USGS_siteno', 'OilLevelDate', 'OilLevelTime']).filter(lambda x: len(x) > 1)
print('For oil levels:', chk.shape[0]) # if zero, there are no duplicates

# r line 271
chk = dfwl2.groupby(['site_no', 'lev_dt', 'lev_tm']).filter(lambda x: len(x) > 1)
print('For water levels:', chk.shape[0])
display(chk.head(3))

For oil levels: 0
For water levels: 0


,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft


In [164]:
# r line 285 Rename columns and select columns in oil levels
oil_levID = tblOilLevels8.rename(columns={'OilLevelDate': 'LevelDate', 'OilLevelTime': 'LevelTime'})
display(oil_levID.head(2))

,NWTPosition_m,LocalUseCode,OilLevelID,LocalSiteName,PersonMakingMeasurement,LevelDate,LevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,...,HIF_ID,GWSI_Tape_Correction_ft,OilLevel_ftBMP_corr,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft,OilLevel_ftBLS,lev_meth_cd,Comments_FluidLevelMethod
0,-4.02,wo,20059,315,A. Berg & J. Lund,2017-06-14,16:01,CDT,22.94,Z,...,HIF-242570,0.0,22.94,473426095052530,1983-07-01,2025-02-21,2.52,20.42,Z,Other
1,27.55,wo,20060,317,A. Berg & J. Lund,2017-06-14,15:25,CDT,31.08,Z,...,HIF-242570,0.0,31.08,473425095052009,1983-07-02,2025-02-21,1.86,29.22,Z,Other


In [165]:
time_check(dfwl2, 'site_no')

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
165,USGS,473413095053701,GW,2023-06-21,15:32,UTC,18.02,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473413095053701,1985-05-16,2025-02-21,2.56
167,USGS,473413095053701,GW,2024-07-25,10:45,UTC,17.73,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473413095053701,1985-05-16,2025-02-21,2.56


In [166]:
# rename columns
wat_levID = dfwl2.rename(columns={'site_no': 'USGS_siteno', 'lev_dt': 'LevelDate', 'lev_tm': 'LevelTime'})
display(wat_levID.head(2))

,agency_cd,USGS_siteno,site_tp_cd,LevelDate,LevelTime,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
0,USGS,473356095043701,ST,2019-06-28,08:56,UTC,0.0,NaN,NaN,NaN,...,D,A,72019,NaN,NaN,Differential Global Positioning System. This c...,473356095043701,2019-06-28,2025-02-21,0.0
1,USGS,473356095043701,ST,2020-08-06,12:00,UTC,-1.6,NaN,NaN,NaN,...,D,A,72019,NaN,NaN,Differential Global Positioning System. This c...,473356095043701,2019-06-28,2025-02-21,0.0


In [167]:
time_check(wat_levID, 'USGS_siteno')

,agency_cd,USGS_siteno,site_tp_cd,LevelDate,LevelTime,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
165,USGS,473413095053701,GW,2023-06-21,15:32,UTC,18.02,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473413095053701,1985-05-16,2025-02-21,2.56
167,USGS,473413095053701,GW,2024-07-25,10:45,UTC,17.73,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473413095053701,1985-05-16,2025-02-21,2.56


In [168]:
# INSPECT
print(oil_levID.keys())
print(oil_levID.shape)
print(wat_levID.keys())
print(wat_levID.shape)

Index(['NWTPosition_m', 'LocalUseCode', 'OilLevelID', 'LocalSiteName',
       'PersonMakingMeasurement', 'LevelDate', 'LevelTime', 'TimeDatum',
       'OilLevel_ftBMP', 'OilLevelMethod', 'Oil_Tape_Serial_No',
       'Comments_OilLevel', 'OilLevelDataSource', 'ZeroThicknessCalcExclude',
       'USGS_siteno', 'AgencyCode', 'XcoordUTMNAD83_m', 'YcoordUTMNAD83_m',
       'OnNWT', 'HIF_ID', 'GWSI_Tape_Correction_ft', 'OilLevel_ftBMP_corr',
       'GWSI_USGS_siteno', 'GWSI_MP_BeginDate', 'GWSI_MP_EndDate',
       'GWSI_MP_height_ft', 'OilLevel_ftBLS', 'lev_meth_cd',
       'Comments_FluidLevelMethod'],
      dtype='object')
(4879, 29)
Index(['agency_cd', 'USGS_siteno', 'site_tp_cd', 'LevelDate', 'LevelTime',
       'lev_tz_cd', 'lev_va', 'sl_lev_va', 'sl_datum_cd', 'lev_status_cd',
       'lev_agency_cd', 'lev_dt_acy_cd', 'lev_acy_cd', 'lev_src_cd',
       'lev_meth_cd', 'lev_age_cd', 'parameter_cd',
       'WebQualifier_GWSIStatusCode', 'Comments_FluidLevelStatus',
       'Comments_FluidLev

In [169]:
# r 303
# make one df with all rows for both water and oil
levID = pd.concat([
    wat_levID[['USGS_siteno', 'LevelDate', 'LevelTime']], 
    oil_levID[['USGS_siteno', 'LevelDate', 'LevelTime']]
    ], 
    ignore_index=True)
# remove dubs
levID = levID.drop_duplicates()
print(levID.shape)
display(levID)



(25905, 3)


,USGS_siteno,LevelDate,LevelTime
0,473356095043701,2019-06-28,08:56
1,473356095043701,2020-08-06,12:00
2,473356095043701,2021-08-19,10:00
3,473356095043701,2022-08-25,08:08
4,473356095043701,2023-06-22,09:30
...,...,...,...
30271,473425095052702,2015-08-05,09:23
30278,473420095052301,2015-08-05,09:36
30283,473426095052526,2015-10-22,11:30
30298,473426095052414,2019-10-31,11:00


In [170]:
display(levID.loc[levID['USGS_siteno']=='473413095053701'].tail(2))

,USGS_siteno,LevelDate,LevelTime
132,473413095053701,2023-06-21,15:32
133,473413095053701,2024-07-25,10:45


In [171]:
# missing lev_va
display(dfwl2.loc[dfwl2['site_no']=='473425095052603'])
display(wat_levID.loc[wat_levID['USGS_siteno']=='473425095052603'])
display(levID.loc[levID['USGS_siteno']=='473425095052603'])
print(levID.loc[levID['USGS_siteno']=='473425095052603'].shape)

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
7441,USGS,473425095052603,GW,2012-06-20,13:46,UTC,24.00,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7443,USGS,473425095052603,GW,2013-07-15,10:38,UTC,23.84,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7445,USGS,473425095052603,GW,2014-07-15,10:14,UTC,23.14,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7447,USGS,473425095052603,GW,2015-08-05,09:04,UTC,24.43,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7449,USGS,473425095052603,GW,2017-06-14,16:04,UTC,24.04,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7451,USGS,473425095052603,GW,2018-06-25,13:30,UTC,24.04,NaN,NaN,8.0,...,O,A,72019,8,Foreign substance was present on the surface o...,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7453,USGS,473425095052603,GW,2019-06-18,13:44,UTC,22.82,NaN,NaN,8.0,...,O,A,72019,8,Foreign substance was present on the surface o...,Observed,473425095052603,2005-06-02,2025-02-21,2.51


,agency_cd,USGS_siteno,site_tp_cd,LevelDate,LevelTime,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
7441,USGS,473425095052603,GW,2012-06-20,13:46,UTC,24.00,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7443,USGS,473425095052603,GW,2013-07-15,10:38,UTC,23.84,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7445,USGS,473425095052603,GW,2014-07-15,10:14,UTC,23.14,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7447,USGS,473425095052603,GW,2015-08-05,09:04,UTC,24.43,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7449,USGS,473425095052603,GW,2017-06-14,16:04,UTC,24.04,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7451,USGS,473425095052603,GW,2018-06-25,13:30,UTC,24.04,NaN,NaN,8.0,...,O,A,72019,8,Foreign substance was present on the surface o...,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7453,USGS,473425095052603,GW,2019-06-18,13:44,UTC,22.82,NaN,NaN,8.0,...,O,A,72019,8,Foreign substance was present on the surface o...,Observed,473425095052603,2005-06-02,2025-02-21,2.51


,USGS_siteno,LevelDate,LevelTime
5019,473425095052603,2012-06-20,13:46
5020,473425095052603,2013-07-15,10:38
5021,473425095052603,2014-07-15,10:14
5022,473425095052603,2015-08-05,09:04
5023,473425095052603,2017-06-14,16:04
5024,473425095052603,2018-06-25,13:30
5025,473425095052603,2019-06-18,13:44
29706,473425095052603,2005-06-27,12:01
30212,473425095052603,2016-07-25,15:09


(9, 3)


In [172]:
# MERGE
out_MasterWaterOilLevels = pd.merge(levID, 
                                    tblSites[["AgencyCode","USGS_siteno","LocalSiteName","XcoordUTMNAD83_m","YcoordUTMNAD83_m","OnNWT","NWTPosition_m"]],
                                    how='left')

out_MasterWaterOilLevels = out_MasterWaterOilLevels.rename(
    columns ={'OnNWT': 'OnNorthWellTransect', 
              'NWTPosition_m': 'DistanceFromCenterOfNorthOilBody_m'})

out_MasterWaterOilLevels

,USGS_siteno,LevelDate,LevelTime,AgencyCode,LocalSiteName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m
0,473356095043701,2019-06-28,08:56,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN
1,473356095043701,2020-08-06,12:00,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN
2,473356095043701,2021-08-19,10:00,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN
3,473356095043701,2022-08-25,08:08,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN
4,473356095043701,2023-06-22,09:30,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN
...,...,...,...,...,...,...,...,...,...
25900,473425095052702,2015-08-05,09:23,USGS,9017,342764.87,5271033.20,1.0,-21.97
25901,473420095052301,2015-08-05,09:36,USGS,1429,342814.32,5270875.34,NaN,NaN
25902,473426095052526,2015-10-22,11:30,USGS,301A,342781.07,5271045.79,1.0,-2.06
25903,473426095052414,2019-10-31,11:00,USGS,534A,342809.72,5271045.08,1.0,23.84


In [173]:
out_MasterWaterOilLevels.loc[out_MasterWaterOilLevels['USGS_siteno']=='473425095052603']

,USGS_siteno,LevelDate,LevelTime,AgencyCode,LocalSiteName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m
5019,473425095052603,2012-06-20,13:46,USGS,0501,342781.18,5271042.47,1.0,-3.31
5020,473425095052603,2013-07-15,10:38,USGS,0501,342781.18,5271042.47,1.0,-3.31
5021,473425095052603,2014-07-15,10:14,USGS,0501,342781.18,5271042.47,1.0,-3.31
5022,473425095052603,2015-08-05,09:04,USGS,0501,342781.18,5271042.47,1.0,-3.31
5023,473425095052603,2017-06-14,16:04,USGS,0501,342781.18,5271042.47,1.0,-3.31
5024,473425095052603,2018-06-25,13:30,USGS,0501,342781.18,5271042.47,1.0,-3.31
5025,473425095052603,2019-06-18,13:44,USGS,0501,342781.18,5271042.47,1.0,-3.31
25861,473425095052603,2005-06-27,12:01,USGS,0501,342781.18,5271042.47,1.0,-3.31
25897,473425095052603,2016-07-25,15:09,USGS,0501,342781.18,5271042.47,1.0,-3.31


In [174]:
'''
# r 390 merge info from bmj3 with master site-date-time list
out_MasterWaterOilLevels2 = pd.merge(out_MasterWaterOilLevels, bmj3_tmp, how='left', left_on='USGS_siteno', right_on='GWSI_USGS_siteno')
# check how many water table wells in tblwells
print(sum(tblWells['WaterTableWell'] == 'Y'))

# r 418: calculate water level elevations and rename columns appropriately
out_MasterWaterOilLevels2 = out_MasterWaterOilLevels2.rename(columns={'lev_va': 'WaterLevel_ftBLS'})
out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] = out_MasterWaterOilLevels2['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - out_MasterWaterOilLevels2['WaterLevel_ftBLS']
out_MasterWaterOilLevels2['WaterLevel_mASL_NAVD88'] = round(out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] * 0.3048, 3)

# r 425 merge with oil levels
# first convert columns to be the same data types and same date format
out_MasterWaterOilLevels2['LevelDate'] = pd.to_datetime(out_MasterWaterOilLevels2['LevelDate']).dt.strftime('%Y-%m-%d')

# r 418: calculate water level elevations and rename columns appropriately
out_MasterWaterOilLevels2 = out_MasterWaterOilLevels2.rename(columns={'lev_va': 'WaterLevel_ftBLS'})
out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] = out_MasterWaterOilLevels2['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - out_MasterWaterOilLevels2['WaterLevel_ftBLS']
out_MasterWaterOilLevels2['WaterLevel_mASL_NAVD88'] = round(out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] * 0.3048, 3)

print(out_MasterWaterOilLevels2.Comments_FluidLevelMethod_oil.unique())

# r 435 merge the fluid level method columns
out_MasterWaterOilLevels2.loc[out_MasterWaterOilLevels2['Comments_FluidLevelMethod_WL'].isna() & ~out_MasterWaterOilLevels2['Comments_FluidLevelMethod_oil'].isna(),'Comments_FluidLevelMethod_WL'] = \
 out_MasterWaterOilLevels2.loc[out_MasterWaterOilLevels2['Comments_FluidLevelMethod_WL'].isna() & ~out_MasterWaterOilLevels2['Comments_FluidLevelMethod_oil'].isna(),'Comments_FluidLevelMethod_oil']
print(out_MasterWaterOilLevels2.Comments_FluidLevelMethod_WL.unique())
# r 440 delete oil fulid level comments column and rename the water fluid level method column
out_MasterWaterOilLevels2['Comments_FluidLevelMethod'] = out_MasterWaterOilLevels2['Comments_FluidLevelMethod_WL']
out_MasterWaterOilLevels2.drop('Comments_FluidLevelMethod_oil', axis=1, inplace=True)
print(out_MasterWaterOilLevels2.Comments_FluidLevelMethod.unique())

# r 452 calculate oil level elevations
out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88'] = out_MasterWaterOilLevels2['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - out_MasterWaterOilLevels2['OilLevel_ftBLS']
out_MasterWaterOilLevels2['OilLevel_mASL_NAVD88'] = out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88'].mul(0.3048).round(3)
# trying new way to achieve same column...
out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88_v2'] = out_MasterWaterOilLevels2.apply(lambda x: x['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - x['OilLevel_ftBLS'], axis=1)

# r 459 calculate oil thicknesses, using the most raw measurements
out_MasterWaterOilLevels2['OilThickness_ft'] = round(out_MasterWaterOilLevels2['WaterLevel_ftBLS'] - out_MasterWaterOilLevels2['OilLevel_ftBLS'], 2)
out_MasterWaterOilLevels2['OilThickness_m'] = round(out_MasterWaterOilLevels2['OilThickness_ft'] * 0.3048, 3)
'''

"\n# r 390 merge info from bmj3 with master site-date-time list\nout_MasterWaterOilLevels2 = pd.merge(out_MasterWaterOilLevels, bmj3_tmp, how='left', left_on='USGS_siteno', right_on='GWSI_USGS_siteno')\n# check how many water table wells in tblwells\nprint(sum(tblWells['WaterTableWell'] == 'Y'))\n\n# r 418: calculate water level elevations and rename columns appropriately\nout_MasterWaterOilLevels2 = out_MasterWaterOilLevels2.rename(columns={'lev_va': 'WaterLevel_ftBLS'})\nout_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] = out_MasterWaterOilLevels2['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - out_MasterWaterOilLevels2['WaterLevel_ftBLS']\nout_MasterWaterOilLevels2['WaterLevel_mASL_NAVD88'] = round(out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] * 0.3048, 3)\n\n# r 425 merge with oil levels\n# first convert columns to be the same data types and same date format\nout_MasterWaterOilLevels2['LevelDate'] = pd.to_datetime(out_MasterWaterOilLevels2['LevelDate']).dt.strftime('%Y-%m-%d')

In [175]:
dfwl2.loc[dfwl2['site_no']=='473425095052603']

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
7441,USGS,473425095052603,GW,2012-06-20,13:46,UTC,24.00,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7443,USGS,473425095052603,GW,2013-07-15,10:38,UTC,23.84,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7445,USGS,473425095052603,GW,2014-07-15,10:14,UTC,23.14,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7447,USGS,473425095052603,GW,2015-08-05,09:04,UTC,24.43,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473425095052603,2005-06-02,2025-02-21,2.51
7449,USGS,473425095052603,GW,2017-06-14,16:04,UTC,24.04,NaN,NaN,1.0,...,O,A,72019,1,Static,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7451,USGS,473425095052603,GW,2018-06-25,13:30,UTC,24.04,NaN,NaN,8.0,...,O,A,72019,8,Foreign substance was present on the surface o...,Observed,473425095052603,2005-06-02,2025-02-21,2.51
7453,USGS,473425095052603,GW,2019-06-18,13:44,UTC,22.82,NaN,NaN,8.0,...,O,A,72019,8,Foreign substance was present on the surface o...,Observed,473425095052603,2005-06-02,2025-02-21,2.51


In [176]:
dfwl2

,agency_cd,site_no,site_tp_cd,lev_dt,lev_tm,lev_tz_cd,lev_va,sl_lev_va,sl_datum_cd,lev_status_cd,...,lev_meth_cd,lev_age_cd,parameter_cd,WebQualifier_GWSIStatusCode,Comments_FluidLevelStatus,Comments_FluidLevelMethod,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft
0,USGS,473356095043701,ST,2019-06-28,08:56,UTC,0.00,NaN,NaN,NaN,...,D,A,72019,NaN,NaN,Differential Global Positioning System. This c...,473356095043701,2019-06-28,2025-02-21,0.00
1,USGS,473356095043701,ST,2020-08-06,12:00,UTC,-1.60,NaN,NaN,NaN,...,D,A,72019,NaN,NaN,Differential Global Positioning System. This c...,473356095043701,2019-06-28,2025-02-21,0.00
2,USGS,473356095043701,ST,2021-08-19,10:00,UTC,-0.16,NaN,NaN,NaN,...,D,A,72019,NaN,NaN,Differential Global Positioning System. This c...,473356095043701,2019-06-28,2025-02-21,0.00
3,USGS,473356095043701,ST,2022-08-25,08:08,UTC,-1.06,NaN,NaN,NaN,...,D,A,72019,NaN,NaN,Differential Global Positioning System. This c...,473356095043701,2019-06-28,2025-02-21,0.00
4,USGS,473356095043701,ST,2023-06-22,09:30,UTC,-1.18,NaN,NaN,NaN,...,D,A,72019,NaN,NaN,Differential Global Positioning System. This c...,473356095043701,2019-06-28,2025-02-21,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36296,USGS,473430095052201,GW,2020-08-06,13:00,UTC,23.33,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473430095052201,2010-03-10,2025-02-21,1.89
36298,USGS,473430095052201,GW,2021-08-19,16:05,UTC,NaN,NaN,NaN,6.0,...,V,A,72019,6,Measurement unable to be obtained due to local...,Calibrated electric tape – accuracy of instrum...,473430095052201,2010-03-10,2025-02-21,1.89
36300,USGS,473430095052201,GW,2022-08-26,13:36,UTC,23.01,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473430095052201,2010-03-10,2025-02-21,1.89
36302,USGS,473430095052201,GW,2023-06-22,14:36,UTC,23.73,NaN,NaN,1.0,...,V,A,72019,1,Static,Calibrated electric tape – accuracy of instrum...,473430095052201,2010-03-10,2025-02-21,1.89


In [177]:
# r 390 merge info from bmj3 with master site-date-time list
out_MasterWaterOilLevels2 = pd.merge(out_MasterWaterOilLevels, bmj3_tmp, how='left', left_on='USGS_siteno', right_on='GWSI_USGS_siteno')
out_MasterWaterOilLevels2.head(3)

#then get the LocalUseCode field from tblWells (in local Access DB),
#check number of WaterTableWells in tblWells (not done algorithmically)
tblWells.keys()

#the water table well field in the database is less comprehensive than the algorithm used above. 
print(sum(tblWells['WaterTableWell'] == 'Y'))

# merge again to get tblWells local use code
out_MasterWaterOilLevels2 = pd.merge(out_MasterWaterOilLevels2, tblWells[['LocalSiteName', 'LocalUseCode']], how='left')
out_MasterWaterOilLevels2.head(2)   

# r line 409 merge to get Water levels
# 'FullName' column is replacing the "Comments_WaterLevelStatus" column
out_MasterWaterOilLevels2 = pd.merge(out_MasterWaterOilLevels2, dfwl2[["site_no","lev_dt","lev_tm",
                                                                       "lev_va", 'Comments_FluidLevelStatus', #"Comments_WaterLevelStatus", 
                                                                       "Comments_FluidLevelMethod","GWSI_MP_height_ft"]],
                                     how='left', 
                                     left_on=['USGS_siteno', "LevelDate", "LevelTime"],
                                     right_on=['site_no', "lev_dt", "lev_tm"])

# r 418: calculate water level elevations and rename columns appropriately
out_MasterWaterOilLevels2 = out_MasterWaterOilLevels2.rename(columns={'lev_va': 'WaterLevel_ftBLS'})
out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] = out_MasterWaterOilLevels2['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - out_MasterWaterOilLevels2['WaterLevel_ftBLS']
out_MasterWaterOilLevels2['WaterLevel_mASL_NAVD88'] = round(out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] * 0.3048, 3)
out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'] = round(out_MasterWaterOilLevels2['WaterLevel_ftASL_NAVD88'], 2)

# r 425 merge with oil levels
# first convert columns to be the same data types and same date format
out_MasterWaterOilLevels2['LevelDate'] = pd.to_datetime(out_MasterWaterOilLevels2['LevelDate']).dt.strftime('%Y-%m-%d')
# MERGE
out_MasterWaterOilLevels2 = pd.merge(out_MasterWaterOilLevels2, tblOilLevels8[["USGS_siteno",
                                                                               "OilLevelDate",
                                                                               "OilLevelTime",
                                                                               "OilLevelID",
                                                                               "PersonMakingMeasurement",
                                                                               "Comments_FluidLevelMethod",
                                                                               "Oil_Tape_Serial_No",
                                                                               "Comments_OilLevel",
                                                                               "GWSI_Tape_Correction_ft",
                                                                               "GWSI_MP_height_ft",
                                                                               "OilLevel_ftBLS"]],
                                     left_on=['USGS_siteno', "LevelDate", "LevelTime"],
                                     right_on=['USGS_siteno', "OilLevelDate", "OilLevelTime"],
                                     how='left',
                                     suffixes=('_WL','_oil'),
                                     indicator='merge_result'
                                     )
chk = out_MasterWaterOilLevels2.copy()
print(out_MasterWaterOilLevels2.Comments_FluidLevelMethod_oil.unique())
# r 435 merge the fluid level method columns
out_MasterWaterOilLevels2.loc[out_MasterWaterOilLevels2['Comments_FluidLevelMethod_WL'].isna() & ~out_MasterWaterOilLevels2['Comments_FluidLevelMethod_oil'].isna(),'Comments_FluidLevelMethod_WL'] = \
 out_MasterWaterOilLevels2.loc[out_MasterWaterOilLevels2['Comments_FluidLevelMethod_WL'].isna() & ~out_MasterWaterOilLevels2['Comments_FluidLevelMethod_oil'].isna(),'Comments_FluidLevelMethod_oil']
print(out_MasterWaterOilLevels2.Comments_FluidLevelMethod_WL.unique())
# r 440 delete oil fulid level comments column and rename the water fluid level method column
out_MasterWaterOilLevels2['Comments_FluidLevelMethod'] = out_MasterWaterOilLevels2['Comments_FluidLevelMethod_WL']
out_MasterWaterOilLevels2.drop('Comments_FluidLevelMethod_oil', axis=1, inplace=True)
print(out_MasterWaterOilLevels2.Comments_FluidLevelMethod.unique())

# r 452 calculate oil level elevations
out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88'] = out_MasterWaterOilLevels2['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - out_MasterWaterOilLevels2['OilLevel_ftBLS']
out_MasterWaterOilLevels2['OilLevel_mASL_NAVD88'] = out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88'].mul(0.3048).round(3)
out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88'] = out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88'].round(2)

# trying new way to achieve same column...
out_MasterWaterOilLevels2['OilLevel_ftASL_NAVD88_v2'] = out_MasterWaterOilLevels2.apply(lambda x: x['GWSI_LandSurfaceAltitude_ftASL_NAVD88'] - x['OilLevel_ftBLS'], axis=1)

# r 459 calculate oil thicknesses, using the most raw measurements
out_MasterWaterOilLevels2['OilThickness_ft'] = round(out_MasterWaterOilLevels2['WaterLevel_ftBLS'] - out_MasterWaterOilLevels2['OilLevel_ftBLS'], 2)
out_MasterWaterOilLevels2['OilThickness_m'] = round(out_MasterWaterOilLevels2['OilThickness_ft'] * 0.3048, 3)


88
[nan 'Steel-tape'
 'Calibrated electric tape – accuracy of instrument has been checked'
 'Other']
['Differential Global Positioning System. This code is especially applicable to surface expressions of groundwater.'
 'Steel-tape'
 'Calibrated electric tape – accuracy of instrument has been checked'
 'Observed' 'Other' 'Electric-tape' nan]
['Differential Global Positioning System. This code is especially applicable to surface expressions of groundwater.'
 'Steel-tape'
 'Calibrated electric tape – accuracy of instrument has been checked'
 'Observed' 'Other' 'Electric-tape' nan]


## Some checks

In [178]:
# r 467 check to see that all oil level IDs are included in final table
# INSPECT
#0 rows indicates all OilLevelID s are in the final merged table
# issue could be related to example of anti join given in my Lession.ipynb file under Libraries > PANDAS > "anti-join"
pd.merge(tblOilLevels8, out_MasterWaterOilLevels2, how='outer', on='OilLevelID', indicator=True).query("_merge != 'both'")

,NWTPosition_m,LocalUseCode_x,OilLevelID,LocalSiteName_x,PersonMakingMeasurement_x,OilLevelDate_x,OilLevelTime_x,TimeDatum,OilLevel_ftBMP,OilLevelMethod,...,GWSI_MP_height_ft_oil,OilLevel_ftBLS_y,merge_result,Comments_FluidLevelMethod_y,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilLevel_ftASL_NAVD88_v2,OilThickness_ft,OilThickness_m,_merge
4879,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Differential Global Positioning System. This c...,NaN,NaN,NaN,NaN,NaN,right_only
4880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Differential Global Positioning System. This c...,NaN,NaN,NaN,NaN,NaN,right_only
4881,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Differential Global Positioning System. This c...,NaN,NaN,NaN,NaN,NaN,right_only
4882,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Differential Global Positioning System. This c...,NaN,NaN,NaN,NaN,NaN,right_only
4883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Differential Global Positioning System. This c...,NaN,NaN,NaN,NaN,NaN,right_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN,right_only
25901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN,right_only
25902,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN,right_only
25903,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN,right_only


In [179]:
# INSPECT
#r 471 check for duplicate in oil level id:
chk = out_MasterWaterOilLevels2[~out_MasterWaterOilLevels2['OilLevelID'].isna()].groupby('OilLevelID').filter(lambda x: len(x) > 1)
print(len(chk), 'rows') # if zero, then no duplicates
display(chk)

0 rows


,USGS_siteno,LevelDate,LevelTime,AgencyCode,LocalSiteName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,GWSI_USGS_siteno,...,GWSI_Tape_Correction_ft,GWSI_MP_height_ft_oil,OilLevel_ftBLS,merge_result,Comments_FluidLevelMethod,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilLevel_ftASL_NAVD88_v2,OilThickness_ft,OilThickness_m


In [180]:
# r 478 check for negative oil thicknesses
NegThk = out_MasterWaterOilLevels2['OilThickness_ft'] <= 0 # note: just retruns boolean mask of true or false for each row of that column
NegThk = out_MasterWaterOilLevels2.loc[out_MasterWaterOilLevels2['OilThickness_ft'] <= 0]
NegThk # if non exist...good
NegThk.to_csv("DataChecks/{}_NegativeOilThicknesses.csv".format(str(datetime.now().date())), index=False)

In [181]:
# INSPECT
# r 485 check to see that oil wells are classified as water table wells
# copied this check into notepad file for Andrew to look into.
WT_oilWell_chk = out_MasterWaterOilLevels2.loc[(out_MasterWaterOilLevels2['WaterTableWell'] == 'N') & (out_MasterWaterOilLevels2['LocalUseCode'] == 'wo')]\
    .groupby('LocalSiteName').size().reset_index(name='count')
display(WT_oilWell_chk)

,LocalSiteName,count
0,0501,9
1,302,155
2,421A,130
3,9014,32
4,9017,31
5,9018,1


In [182]:
# INSPECT more checks for oil thickness calcs

# number of thickness values
print(out_MasterWaterOilLevels2[~out_MasterWaterOilLevels2['OilThickness_ft'].isna()]['OilThickness_ft'].count())

# number of oil level values
print(out_MasterWaterOilLevels2[~out_MasterWaterOilLevels2['OilLevel_ftBLS'].isna()]['OilLevel_ftBLS'].count())

# number of water levels
print(out_MasterWaterOilLevels2[~out_MasterWaterOilLevels2['WaterLevel_ftBLS'].isna()]['WaterLevel_ftBLS'].count())
print(dfwl2[~dfwl2['lev_va'].isna()]['lev_va'].count())

3151
3553
25103
25103


In [183]:
# INSPECT
# check count of each column and inspect which rows have valid values
out_MasterWaterOilLevels2.count()
out_MasterWaterOilLevels2[~out_MasterWaterOilLevels2['Comments_FluidLevelStatus'].isna()] # not many rows available for this column

,USGS_siteno,LevelDate,LevelTime,AgencyCode,LocalSiteName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,GWSI_USGS_siteno,...,GWSI_Tape_Correction_ft,GWSI_MP_height_ft_oil,OilLevel_ftBLS,merge_result,Comments_FluidLevelMethod,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilLevel_ftASL_NAVD88_v2,OilThickness_ft,OilThickness_m
6,473358095061401,1987-05-05,12:01,USGS,426,341759.50,5270225.09,NaN,NaN,473358095061401,...,NaN,NaN,NaN,left_only,Steel-tape,NaN,NaN,NaN,NaN,NaN
7,473358095061401,1989-06-01,12:01,USGS,426,341759.50,5270225.09,NaN,NaN,473358095061401,...,NaN,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN
8,473358095061401,1992-06-16,12:01,USGS,426,341759.50,5270225.09,NaN,NaN,473358095061401,...,NaN,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN
9,473404095054101,1984-10-15,12:01,USGS,409,342437.61,5270398.26,NaN,NaN,473404095054101,...,NaN,NaN,NaN,left_only,Steel-tape,NaN,NaN,NaN,NaN,NaN
10,473404095054101,1984-10-31,12:01,USGS,409,342437.61,5270398.26,NaN,NaN,473404095054101,...,NaN,NaN,NaN,left_only,Steel-tape,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25425,473430095052201,2020-08-06,13:00,USGS,9019,342831.24,5271157.77,NaN,NaN,473430095052201,...,NaN,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN
25426,473430095052201,2021-08-19,16:05,USGS,9019,342831.24,5271157.77,NaN,NaN,473430095052201,...,NaN,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN
25427,473430095052201,2022-08-26,13:36,USGS,9019,342831.24,5271157.77,NaN,NaN,473430095052201,...,NaN,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN
25428,473430095052201,2023-06-22,14:36,USGS,9019,342831.24,5271157.77,NaN,NaN,473430095052201,...,NaN,NaN,NaN,left_only,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN


In [184]:
# r 521 create a dataset ID column
out_MasterWaterOilLevels2['dataset_id'] = 'ds_12'

In [185]:
# INSPECTION
display(tblOilLevels8[["LocalSiteName","OilLevelDate","OilLevelTime","ZeroThicknessCalcExclude"]].dtypes)
#out_MasterWaterOilLevels2[['LocalSiteName','LevelDate','LevelTime']].
out_MasterWaterOilLevels2.LevelDate

LocalSiteName               object
OilLevelDate                object
OilLevelTime                object
ZeroThicknessCalcExclude    object
dtype: object

0        2019-06-28
1        2020-08-06
2        2021-08-19
3        2022-08-25
4        2023-06-22
            ...    
25900    2015-08-05
25901    2015-08-05
25902    2015-10-22
25903    2019-10-31
25904    2005-07-15
Name: LevelDate, Length: 25905, dtype: object

In [186]:
# r 525
#calculate 0 oil thickness for all wells with water but not oil, except for 
#records where water was not measured (ZeroThicknessCalcExclude) column. 

#merge master file to get "excludeFromZeroThicknessCalc from tblOilLevels (tblOilLevels8 in Python)
# to merge, first we have to match data types
tblOilLevels8['OilLevelDate'] = pd.to_datetime(tblOilLevels8['OilLevelDate']).dt.strftime('%Y-%m-%d')

out_MasterWaterOilLevels3 = pd.merge(out_MasterWaterOilLevels2, tblOilLevels8[["LocalSiteName","OilLevelDate","OilLevelTime","ZeroThicknessCalcExclude"]], 
                                     left_on=['LocalSiteName','LevelDate','LevelTime'],
                                     right_on=['LocalSiteName','OilLevelDate','OilLevelTime'],
                                     how='left')
# couple checks to see if the merge worked r 535
print(out_MasterWaterOilLevels2.shape[0])
print(out_MasterWaterOilLevels3.shape[0])
print(out_MasterWaterOilLevels3[~out_MasterWaterOilLevels3['ZeroThicknessCalcExclude'].isna()].shape[0])   # these two do the same thing
print(out_MasterWaterOilLevels3[pd.notna(out_MasterWaterOilLevels3['ZeroThicknessCalcExclude'])].shape[0]) # these two do the same thing
print(tblOilLevels8[~tblOilLevels8['ZeroThicknessCalcExclude'].isna()].shape[0])

25905
25905
16
16
16


In [187]:
# r 542 more checks
print(out_MasterWaterOilLevels3[(out_MasterWaterOilLevels3['WaterLevel_mASL_NAVD88'] > 0) & (out_MasterWaterOilLevels3['OilLevel_ftBLS'].isna())].shape[0]) 
print(out_MasterWaterOilLevels3[(out_MasterWaterOilLevels3['WaterLevel_mASL_NAVD88'] > 0) & (out_MasterWaterOilLevels3['OilLevel_mASL_NAVD88'].isna())].shape[0])
print(out_MasterWaterOilLevels3[(out_MasterWaterOilLevels3['WaterLevel_mASL_NAVD88'] > 0) & 
                                (out_MasterWaterOilLevels3['OilLevel_mASL_NAVD88'].isna()) &
                                (out_MasterWaterOilLevels3['ZeroThicknessCalcExclude'].isna()) # same as print statement before except with this added conditional
                                ].shape[0])
# r 551 if there is water, and no oil, set oil thickness to 0
out_MasterWaterOilLevels3.loc[(out_MasterWaterOilLevels3['WaterLevel_mASL_NAVD88'] > 0) &
                              (out_MasterWaterOilLevels3['OilLevel_mASL_NAVD88'].isna()) &
                              (out_MasterWaterOilLevels3['ZeroThicknessCalcExclude'].isna()), # picks rows where these conditions are true
                              'OilThickness_m'] = 0                                           # for the OilThickness_m col and replaces it with zero
# repeat but for the ft columns (note: the R script still used the m for the conditions while changing the OilThickness_ft)
out_MasterWaterOilLevels3.loc[(out_MasterWaterOilLevels3['WaterLevel_ftASL_NAVD88'] > 0) & 
                              (out_MasterWaterOilLevels3['OilLevel_ftASL_NAVD88'].isna()) &
                              (out_MasterWaterOilLevels3['ZeroThicknessCalcExclude'].isna()), 
                              'OilThickness_ft'] = 0

out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['ZeroThicknessCalcExclude'].isna()]

21952
21952
21936


,USGS_siteno,LevelDate,LevelTime,AgencyCode,LocalSiteName,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,GWSI_USGS_siteno,...,Comments_FluidLevelMethod,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilLevel_ftASL_NAVD88_v2,OilThickness_ft,OilThickness_m,dataset_id,OilLevelDate_y,OilLevelTime_y,ZeroThicknessCalcExclude
0,473356095043701,2019-06-28,08:56,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN,473356095043701,...,Differential Global Positioning System. This c...,NaN,NaN,NaN,0.0,0.0,ds_12,NaN,NaN,NaN
1,473356095043701,2020-08-06,12:00,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN,473356095043701,...,Differential Global Positioning System. This c...,NaN,NaN,NaN,0.0,0.0,ds_12,NaN,NaN,NaN
2,473356095043701,2021-08-19,10:00,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN,473356095043701,...,Differential Global Positioning System. This c...,NaN,NaN,NaN,0.0,0.0,ds_12,NaN,NaN,NaN
3,473356095043701,2022-08-25,08:08,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN,473356095043701,...,Differential Global Positioning System. This c...,NaN,NaN,NaN,0.0,0.0,ds_12,NaN,NaN,NaN
4,473356095043701,2023-06-22,09:30,USGS,Grant Creek 01,343765.96,5270091.98,NaN,NaN,473356095043701,...,Differential Global Positioning System. This c...,NaN,NaN,NaN,0.0,0.0,ds_12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25900,473425095052702,2015-08-05,09:23,USGS,9017,342764.87,5271033.20,1.0,-21.97,473425095052702,...,Other,1391.07,423.998,1391.07,NaN,NaN,ds_12,2015-08-05,09:23,None
25901,473420095052301,2015-08-05,09:36,USGS,1429,342814.32,5270875.34,NaN,NaN,473420095052301,...,Other,NaN,NaN,NaN,NaN,NaN,ds_12,2015-08-05,09:36,None
25902,473426095052526,2015-10-22,11:30,USGS,301A,342781.07,5271045.79,1.0,-2.06,473426095052526,...,Other,1390.87,423.937,1390.87,NaN,NaN,ds_12,2015-10-22,11:30,None
25903,473426095052414,2019-10-31,11:00,USGS,534A,342809.72,5271045.08,1.0,23.84,473426095052414,...,Calibrated electric tape – accuracy of instrum...,NaN,NaN,NaN,NaN,NaN,ds_12,2019-10-31,11:00,None


In [188]:
# see if there are fewer NAs (note...generally there are just too many NAs)
print(out_MasterWaterOilLevels2.loc[out_MasterWaterOilLevels2['OilThickness_m'].isna()].shape[0])
print(out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['OilThickness_m'].isna()].shape[0])

22754
818


## create estimated water level column for rows where an oil level was measured
calculate apparent water table elevation, based on Geoff's reference in SIR,

.856 is the specific gravity of crude oil.

In [189]:
# r line 569 populate Est water level in m with water levels from water level_m column for records with a water level but oil thickness = NA
out_MasterWaterOilLevels3.loc[(~out_MasterWaterOilLevels3['WaterLevel_mASL_NAVD88'].isna()) & 
                              (out_MasterWaterOilLevels3['OilThickness_m'].isna()),
                              'EstWaterLevel_mASL_NAV88'] = \
                              out_MasterWaterOilLevels3.loc[(~out_MasterWaterOilLevels3['WaterLevel_mASL_NAVD88'].isna()) & 
                                                            (out_MasterWaterOilLevels3['OilThickness_m'].isna()), 
                                                            'WaterLevel_mASL_NAVD88']
# r line 573 populate Est water level in ft with water levels from water level, ft column for records with a water level but oil thickness = NA
out_MasterWaterOilLevels3.loc[(~out_MasterWaterOilLevels3['WaterLevel_ftASL_NAVD88'].isna()) & 
                              (out_MasterWaterOilLevels3['OilThickness_ft'].isna()),
                              'EstWaterLevel_ftASL_NAV88'] = \
                              out_MasterWaterOilLevels3.loc[(~out_MasterWaterOilLevels3['WaterLevel_ftASL_NAVD88'].isna()) & 
                                                            (out_MasterWaterOilLevels3['OilThickness_ft'].isna()), 
                                                            'WaterLevel_ftASL_NAVD88']
# r 578 populate est water level in m column with water level+ oil_thickness*0.855 (Lundy, 2015) for records where 
# a water level and oil thickness is present; 0 oil thickness should produce the 
# an estimated water level identical to the original water level data. 
out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['OilThickness_m'].notna(), 'EstWaterLevel_mASL_NAVD88'] = \
round(out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['OilThickness_m'].notna(), 'WaterLevel_mASL_NAVD88'] +\
      out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['OilThickness_m'].notna(), 'OilThickness_m']*0.855, 3)
# r 581 populate est water level in ft column with water level+ oil_thickness*0.855 for records in ft where a 
# water level and oil thickness is present; 0 oil thickness should produce the 
# an estimated water level identical to the original water level data. 
out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['OilThickness_ft'].notna(), 'EstWaterLevel_ftASL_NAVD88'] = \
round(out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['OilThickness_ft'].notna(), 'WaterLevel_ftASL_NAVD88'] +\
      out_MasterWaterOilLevels3.loc[out_MasterWaterOilLevels3['OilThickness_ft'].notna(), 'OilThickness_ft']*0.855, 3)
# r 586 replace NA in Comments_FluidLevelStatus column with "static" for appropriate conditions
out_MasterWaterOilLevels3.loc[(out_MasterWaterOilLevels3['WaterLevel_ftBLS'].notna()) & 
                              (out_MasterWaterOilLevels3['Comments_FluidLevelStatus'].isna()),
                              'Comments_FluidLevelStatus'] = 'Static'

In [190]:
out_MasterWaterOilLevels3[['USGS_siteno','LocalSiteName','Comments_FluidLevelStatus']]

,USGS_siteno,LocalSiteName,Comments_FluidLevelStatus
0,473356095043701,Grant Creek 01,Static
1,473356095043701,Grant Creek 01,Static
2,473356095043701,Grant Creek 01,Static
3,473356095043701,Grant Creek 01,Static
4,473356095043701,Grant Creek 01,Static
...,...,...,...
25900,473425095052702,9017,NaN
25901,473420095052301,1429,NaN
25902,473426095052526,301A,NaN
25903,473426095052414,534A,NaN


## Export to CSV

In [191]:
# r 590 reorder the columns for the final output
out_MasterWaterOilLevels3 = out_MasterWaterOilLevels3.loc[:, ['dataset_id',
                                                              'AgencyCode',
                                                              "USGS_siteno",
                                                              "LocalSiteName",
                                                              "LocalUseCode", 
                                                              "WaterTableWell",                       
                                                              "XcoordUTMNAD83_m",
                                                              "YcoordUTMNAD83_m",                      
                                                              "OnNorthWellTransect",                  
                                                              "DistanceFromCenterOfNorthOilBody_m",    
                                                              "GWSI_LandSurfaceAltitude_ftASL_NAVD88",
                                                              "GWSI_LandSurfaceAltitude_mASL_NAVD88",
                                                              "MidOfScreenDepth_ft",
                                                              "MidOfScreenDepth_m",
                                                              "MidOfScreenElevation_ftASL_NAVD88",#worked
                                                              "MidOfScreenElevation_mASL_NAVD88",
                                                              "LevelDate", 
                                                              "LevelTime",
                                                              "WaterLevel_ftBLS",
                                                              "Comments_FluidLevelStatus",
                                                              "Comments_FluidLevelMethod",
                                                              "GWSI_MP_height_ft_WL",
                                                              "WaterLevel_ftASL_NAVD88",
                                                              "WaterLevel_mASL_NAVD88",
                                                              "OilLevelID",                           
                                                              "PersonMakingMeasurement", #worked
                                                              "Oil_Tape_Serial_No",
                                                              "GWSI_Tape_Correction_ft",
                                                              "GWSI_MP_height_ft_oil",
                                                              "OilLevel_ftBLS",
                                                              "OilLevel_ftASL_NAVD88",
                                                              "OilLevel_mASL_NAVD88",
                                                              "OilThickness_ft",
                                                              "OilThickness_m",
                                                              "EstWaterLevel_ftASL_NAVD88",
                                                              "EstWaterLevel_mASL_NAVD88",
                                                              "Comments_OilLevel",
                                                              "ZeroThicknessCalcExclude"]]

In [192]:
# save the output
out_MasterWaterOilLevels3.to_csv("outputs/{}_out_MasterOilLevels3.csv".format(str(datetime.now().date())), index=False, encoding='windows-1252')
print('Saved near this dir:', os.getcwd())

Saved near this dir: P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels


In [193]:
# r 638 export master file for data release, just select a subset of columns.
# the site info data release will have all of the location and screen information
# add an "x" prefix to the local site name column
out_MasterWaterOilLevels_DataRelease = out_MasterWaterOilLevels3.loc[:,['dataset_id',
                                                                        "AgencyCode",
                                                                        "USGS_siteno",
                                                                        "LocalSiteName",
                                                                        "LevelDate", 
                                                                        "LevelTime",
                                                                        "WaterLevel_ftBLS",
                                                                        "Comments_FluidLevelStatus",
                                                                        "Comments_FluidLevelMethod",
                                                                        "WaterLevel_ftASL_NAVD88",
                                                                        "WaterLevel_mASL_NAVD88",
                                                                        "OilLevelID",                          
                                                                        "OilLevel_ftBLS",
                                                                        "OilLevel_ftASL_NAVD88",
                                                                        "OilLevel_mASL_NAVD88",
                                                                        "OilThickness_ft",
                                                                        "OilThickness_m",
                                                                        "EstWaterLevel_ftASL_NAVD88",
                                                                        "EstWaterLevel_mASL_NAVD88",
                                                                        "Comments_OilLevel"]]
out_MasterWaterOilLevels_DataRelease['LocalSiteName'] = 'x' + out_MasterWaterOilLevels_DataRelease['LocalSiteName'].astype(str)
out_MasterWaterOilLevels_DataRelease['USGS_siteno'] = 'x' + out_MasterWaterOilLevels_DataRelease['USGS_siteno'].astype(str)
out_MasterWaterOilLevels_DataRelease['OilLevel_ftBLS'] = round(out_MasterWaterOilLevels_DataRelease['OilLevel_ftBLS'], 2)

In [194]:
# save the output
out_MasterWaterOilLevels_DataRelease.to_csv("outputs/{}_tblds_12_WaterAndOilLevels.csv".format(str(datetime.now().date())), index=False, encoding='windows-1252')
print('Saved near this dir:', os.getcwd())

Saved near this dir: P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels


In [195]:
#x = pd.read_csv(os.getcwd() + f"/outputs/2024-10-24_tblds_12_WaterAndOilLevels.csv")
x = pd.read_csv(os.getcwd() + f"/outputs/2024-10-24_out_MasterOilLevels3.csv")
x

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\2141431135.py:2: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  x = pd.read_csv(os.getcwd() + f"/outputs/2024-10-24_out_MasterOilLevels3.csv")


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LocalUseCode,WaterTableWell,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,GWSI_MP_height_ft_oil,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,ZeroThicknessCalcExclude
0,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1390.17,423.724,NaN,NaN
1,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.77,424.211,NaN,NaN
2,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1390.33,423.773,NaN,NaN
3,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.23,424.047,NaN,NaN
4,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.35,424.083,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25894,ds_12,USGS,473425095052702,9017,wo,N,342764.87,5271033.20,1.0,-21.97,...,1.38,20.30,1391.07,423.998,NaN,NaN,NaN,NaN,"No water in well, possible bad screen. Oil to ...",NaN
25895,ds_12,USGS,473420095052301,1429,wo,Y,342814.32,5270875.34,NaN,NaN,...,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No oil detected. Trace of oil on probe. WL mea...,NaN
25896,ds_12,USGS,473426095052526,301A,wo,Y,342781.07,5271045.79,1.0,-2.06,...,1.00,22.81,1390.87,423.937,NaN,NaN,NaN,NaN,Obstruction in well is a bailer at 7.527 meter...,NaN
25897,ds_12,USGS,473426095052414,534A,wo,Y,342809.72,5271045.08,1.0,23.84,...,1.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A sampler was deployed in the well during this...,NaN


In [196]:
#y = pd.read_csv(os.getcwd() + f"/outputs/2024-07-19_tblds_12_WaterAndOilLevels.csv")
y = pd.read_csv(os.getcwd() + f"/outputs/2024-07-19_out_MasterOilLevels3.csv")
y

C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\671195830.py:2: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  y = pd.read_csv(os.getcwd() + f"/outputs/2024-07-19_out_MasterOilLevels3.csv")


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LocalUseCode,WaterTableWell,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,GWSI_MP_height_ft_oil,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,ZeroThicknessCalcExclude
0,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1390.17,423.724,NaN,NaN
1,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.77,424.211,NaN,NaN
2,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1390.33,423.773,NaN,NaN
3,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.23,424.047,NaN,NaN
4,ds_12,USGS,473356095043701,Grant Creek 01,swe,Y,343765.96,5270091.98,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.35,424.083,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25582,ds_12,USGS,473425095052702,9017,wo,N,342764.87,5271033.20,1.0,-21.97,...,1.38,20.30,1391.07,423.998,NaN,NaN,NaN,NaN,"No water in well, possible bad screen. Oil to ...",NaN
25583,ds_12,USGS,473420095052301,1429,wo,Y,342814.32,5270875.34,NaN,NaN,...,2.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No oil detected. Trace of oil on probe. WL mea...,NaN
25584,ds_12,USGS,473426095052526,301A,wo,Y,342781.07,5271045.79,1.0,-2.06,...,1.00,22.81,1390.87,423.937,NaN,NaN,NaN,NaN,Obstruction in well is a bailer at 7.527 meter...,NaN
25585,ds_12,USGS,473426095052414,534A,wo,Y,342809.72,5271045.08,1.0,23.84,...,1.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A sampler was deployed in the well during this...,NaN


In [197]:
test = x[x['LocalSiteName'] == '501A']
test.head()

,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LocalUseCode,WaterTableWell,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,GWSI_MP_height_ft_oil,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,ZeroThicknessCalcExclude
10538,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.74,422.983,NaN,NaN
10539,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.78,422.995,NaN,NaN
10540,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.78,422.995,NaN,NaN
10541,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.81,423.004,NaN,NaN
10542,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.83,423.011,NaN,NaN


In [198]:
test2 = y[y['LocalSiteName'] == '501A']
test2.head()

,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LocalUseCode,WaterTableWell,XcoordUTMNAD83_m,YcoordUTMNAD83_m,OnNorthWellTransect,DistanceFromCenterOfNorthOilBody_m,...,GWSI_MP_height_ft_oil,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,ZeroThicknessCalcExclude
18644,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.74,422.983,NaN,NaN
18645,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.78,422.995,NaN,NaN
18646,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.78,422.995,NaN,NaN
18647,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.81,423.004,NaN,NaN
18648,ds_12,USGS,473428095050802,501A,ww,N,343161.77,5271119.45,1.0,375.83,...,NaN,NaN,NaN,NaN,0.0,0.0,1387.83,423.011,NaN,NaN


# Read in a file to check

In [199]:
# directory
dir = r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs'
# today's date YYYY-MM-DD
date = '2025-02-04'
# file 
file_path = f'{dir}\{date}_tblds_12_WaterAndOilLevels.csv'
# read into csv
chk = pd.read_csv(file_path, encoding='windows-1252')

<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
<string>:6: SyntaxWarning: invalid escape sequence '\{'
<>:6: SyntaxWarning: invalid escape sequence '\{'
C:\Users\bmilinic\AppData\Local\Temp\1\ipykernel_19780\4167333592.py:6: SyntaxWarning: invalid escape sequence '\{'
  file_path = f'{dir}\{date}_tblds_12_WaterAndOilLevels.csv'


In [200]:
chk

,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,WaterLevel_mASL_NAVD88,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel
0,ds_12,USGS,x473356095043701,xGrant Creek 01,2019-06-28,08:56,0.00,static,Differential Global Positioning System. This c...,1390.17,423.724,NaN,NaN,NaN,NaN,0.0,0.0,1390.17,423.724,NaN
1,ds_12,USGS,x473356095043701,xGrant Creek 01,2020-08-06,12:00,-1.60,static,Differential Global Positioning System. This c...,1391.77,424.211,NaN,NaN,NaN,NaN,0.0,0.0,1391.77,424.211,NaN
2,ds_12,USGS,x473356095043701,xGrant Creek 01,2021-08-19,10:00,-0.16,static,Differential Global Positioning System. This c...,1390.33,423.773,NaN,NaN,NaN,NaN,0.0,0.0,1390.33,423.773,NaN
3,ds_12,USGS,x473356095043701,xGrant Creek 01,2022-08-25,08:08,-1.06,static,Differential Global Positioning System. This c...,1391.23,424.047,NaN,NaN,NaN,NaN,0.0,0.0,1391.23,424.047,NaN
4,ds_12,USGS,x473356095043701,xGrant Creek 01,2023-06-22,09:30,-1.18,static,Differential Global Positioning System. This c...,1391.35,424.083,NaN,NaN,NaN,NaN,0.0,0.0,1391.35,424.083,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25900,ds_12,USGS,x473425095052702,x9017,2015-08-05,09:23,NaN,NaN,Other,NaN,NaN,19922.0,20.30,1391.07,423.998,NaN,NaN,NaN,NaN,"No water in well, possible bad screen. Oil to ..."
25901,ds_12,USGS,x473420095052301,x1429,2015-08-05,09:36,NaN,NaN,Other,NaN,NaN,19929.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No oil detected. Trace of oil on probe. WL mea...
25902,ds_12,USGS,x473426095052526,x301A,2015-10-22,11:30,NaN,NaN,Other,NaN,NaN,19934.0,22.81,1390.87,423.937,NaN,NaN,NaN,NaN,Obstruction in well is a bailer at 7.527 meter...
25903,ds_12,USGS,x473426095052414,x534A,2019-10-31,11:00,NaN,NaN,Calibrated electric tape – accuracy of instrum...,NaN,NaN,20215.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A sampler was deployed in the well during this...


# Extra code

In [265]:
stop in the name of love

SyntaxError: invalid syntax (601133638.py, line 1)

In [ ]:
# Example code
bmjmp2['GWSI_USGS_siteno'] = bmjmp2['GWSI_USGS_siteno'].astype(str)  # convert column to strings
tblOilLevels7.loc[:, ['OilLevelDate','GWSI_MP_BeginDate','GWSI_MP_EndDate']].sample(3) # inspect specific columns for random rows

# Good example merge

tblOilLevels7 = pd.merge(tblOilLevels6, bmjmp2.loc[:, ['GWSI_USGS_siteno', 'GWSI_MP_BeginDate', 'GWSI_MP_EndDate', 'GWSI_MP_height_ft']], how='left', left_on='USGS_siteno', right_on='GWSI_USGS_siteno')

'''
dfc = pd.merge(dfa, dfb, how='left', left_on='Col', right_on='Col')

# or select specific columns within dfb to merge into dfa using loc

dfb.loc[:, ['Col1', 'Col2']]

'''

## R vs Python

In [ ]:
print(os.getcwd())

P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels


In [ ]:
# Easily read in py output
file_path = r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\2024-07-19_tblds_12_WaterAndOilLevels.csv"
dfpy = pd.read_csv(file_path)

In [364]:
dfpy = out_MasterWaterOilLevels_DataRelease.copy()

In [365]:
# Read in R output after investigating the encoding

import chardet
file_path = r"P:\0083\analysis\DataCompilation\DataCompilationR\CreateMasterWaterOilLevelDataSet\outputs\2021-04-27_tblds_12_WaterAndOilLevels.csv"
file_path = r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\EXAMPLE_tblds_12_WaterAndOilLevels.csv"

# (r)ead a file's (b)inary
file = open(file_path, 'rb')
# store as raw data
raw_data = file.read()
# close the file
file.close()
# check encoding with chardet
result = chardet.detect(raw_data)
print("Result: ", result)
encoding = result['encoding']
print(f"Detected potential encoding: {encoding}")

# use the encoding to read the csv
dfr = pd.read_csv(file_path, encoding=encoding)

# if reading in the EXAMPLE file skip the first two rows 
if file_path == r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\EXAMPLE_tblds_12_WaterAndOilLevels.csv":
    print('Skipping first two rows for EXAMPLE data')
    dfr = pd.read_csv(file_path, encoding=encoding, skiprows=2)

Result:  {'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
Detected potential encoding: Windows-1252
Skipping first two rows for EXAMPLE data


In [366]:
dfpy['origin'] = 'python'
dfr['origin'] = 'R'

# update r column to match python
#dfr['LocalSiteName'] = 'x' + dfr['LocalSiteName'].astype(str)

In [367]:
dfcompare = pd.concat([dfpy, dfr], axis=0)
dfcompare.keys()

Index(['dataset_id', 'AgencyCode', 'USGS_siteno', 'LocalSiteName', 'LevelDate',
       'LevelTime', 'WaterLevel_ftBLS', 'Comments_WaterLevelStatus',
       'Comments_FluidLevelMethod', 'WaterLevel_ftASL_NAVD88',
       'WaterLevel_mASL_NAVD88', 'OilLevelID', 'OilLevel_ftBLS',
       'OilLevel_ftASL_NAVD88', 'OilLevel_mASL_NAVD88', 'OilThickness_ft',
       'OilThickness_m', 'EstWaterLevel_ftASL_NAVD88',
       'EstWaterLevel_mASL_NAVD88', 'Comments_OilLevel', 'origin'],
      dtype='object')

In [368]:
#dfcompare.sort_values(by=['LocalSiteName','LevelTime', 'WaterLevel_ftBLS'], inplace=True)
dfcompare.sort_values(by=['LocalSiteName', 'WaterLevel_ftBLS', 'LevelTime'], inplace=True)
dfcompare[['LocalSiteName', 'LevelDate','LevelTime','WaterLevel_ftBLS', 'OilLevel_ftBLS', 'origin']]

,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,OilLevel_ftBLS,origin
10289,x0501,2019-06-18,14:44,22.82,NaN,python
9794,x0501,6/18/2019,14:44,22.82,18.99,R
10285,x0501,2014-07-15,11:14,23.14,NaN,python
9790,x0501,7/15/2014,11:14,23.14,19.27,R
10284,x0501,2013-07-15,11:38,23.84,NaN,python
...,...,...,...,...,...,...
24441,xWL04,2022-08-25,13:16,-0.31,NaN,python
23111,xWL04,6/27/2019,17:00,0.00,NaN,R
24439,xWL04,2019-06-27,17:34,0.00,NaN,python
24442,xWL04,2023-06-22,10:20,0.18,NaN,python


In [369]:
dfcompare.sort_values(by=['LocalSiteName','LevelTime', 'WaterLevel_ftBLS'], inplace=True)
#dfcompare.sort_values(by=['LocalSiteName', 'WaterLevel_ftBLS', 'LevelTime'], inplace=True)
chk = dfcompare[['LocalSiteName','USGS_siteno', 'LevelDate','LevelTime','WaterLevel_ftBLS', 'OilLevel_ftBLS', 'origin']].copy()

In [371]:
print(chk.loc[10286, 'OilLevel_ftBLS'])

10286   NaN
10286   NaN
Name: OilLevel_ftBLS, dtype: float64


In [ ]:
# save the output
dfcompare.to_csv(r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\tests\compare.csv')

### Mutli-compare

In [ ]:
# compare 3 dfs
# Easily read in py output
file_path = r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\2024-07-02_tblds_12_WaterAndOilLevels.csv"
dfpy_updated = pd.read_csv(file_path)

In [ ]:
dfpy_updated['origin'] = 'python_updated'

In [ ]:
dfcompare = pd.concat([dfpy, dfpy_updated, dfr], axis=0)
dfcompare.keys()

Index(['dataset_id', 'AgencyCode', 'USGS_siteno', 'LocalSiteName', 'LevelDate',
       'LevelTime', 'WaterLevel_ftBLS', 'Comments_WaterLevelStatus',
       'Comments_FluidLevelMethod', 'WaterLevel_ftASL_NAVD88',
       'WaterLevel_mASL_NAVD88', 'OilLevelID', 'OilLevel_ftBLS',
       'OilLevel_ftASL_NAVD88', 'OilLevel_mASL_NAVD88', 'OilThickness_ft',
       'OilThickness_m', 'EstWaterLevel_ftASL_NAVD88',
       'EstWaterLevel_mASL_NAVD88', 'Comments_OilLevel', 'origin'],
      dtype='object')

In [ ]:
dfcompare.sort_values(by=['LocalSiteName', 'LevelTime','WaterLevel_ftBLS'], inplace=True)

## Explore water and oil
- dfwl2 -> wat_levID
- tblOilLevels8 -> oil_levID
- out_MasterWaterOilLevels3 combines the two

For site 0501
- Yes, TimeDatum = CDT

In [526]:
chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']=='2015-08-05') & (chk['LocalSiteName']=='0501')]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']=='473425095052603')&(chk['lev_dt']=='2015-08-05')]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']=='2015-08-05') & (dfcompare['LocalSiteName']=='x0501')].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
19967,19930,0501,"A. Berg, E. Coenen",2015-08-05,10:04,CDT,23.06,Z,2939,Oil is likely cut off from screen. Extremly da...,...,NaN,NaN,Z,NaN,NaN,NaN,NaN,NaN,None,Oil is likely cut off from screen. Extremly da...


,site_no,lev_dt,lev_tm,lev_va
10286,473425095052603,2015-08-05,09:04,24.43


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
10286,ds_12,USGS,x473425095052603,x0501,2015-08-05,09:04,24.43,Water level was affected by atmospheric pressure.,Calibrated electric tape – accuracy of instrum...,1387.4,...,19930.0,20.55,1391.28,424.062,3.88,1.183,1390.717,423.891,Oil is likely cut off from screen. Extremly da...,python
9791,ds_12,USGS,473425000000000.0,x0501,2015-08-05,10:04,24.43,static,Calibrated electric tape – accuracy of instrum...,1387.4,...,19930.0,20.55,1391.28,424.062,3.88,1.183,1390.720,423.891,Oil is likely cut off from screen. Extremly da...,R


In [527]:
# set up vars
my_date = '2015-08-05'
my_site_local = '0501'
my_site_long = '473425095052603'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
19967,19930,0501,"A. Berg, E. Coenen",2015-08-05,10:04,CDT,23.06,Z,2939,Oil is likely cut off from screen. Extremly da...,...,NaN,NaN,Z,NaN,NaN,NaN,NaN,NaN,None,Oil is likely cut off from screen. Extremly da...


,site_no,lev_dt,lev_tm,lev_va
10286,473425095052603,2015-08-05,09:04,24.43


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
10286,ds_12,USGS,x473425095052603,x0501,2015-08-05,09:04,24.43,Water level was affected by atmospheric pressure.,Calibrated electric tape – accuracy of instrum...,1387.4,...,19930.0,20.55,1391.28,424.062,3.88,1.183,1390.717,423.891,Oil is likely cut off from screen. Extremly da...,python
9791,ds_12,USGS,473425000000000.0,x0501,2015-08-05,10:04,24.43,static,Calibrated electric tape – accuracy of instrum...,1387.4,...,19930.0,20.55,1391.28,424.062,3.88,1.183,1390.720,423.891,Oil is likely cut off from screen. Extremly da...,R


301A 
- Time Datum = None :( 

In [528]:
# set up vars
my_date = '2011-02-03'
my_site_local = '301A'
my_site_long = '473426095052526'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
847,627,301A,A. Berg,2011-02-03,14:00,None,22.26,Z,None,changed time from 1201 to match GWSI water lev...,...,NaN,NaN,Z,430.713,423.568,423.928,6.78,6.42,None,changed time from 1201 to match GWSI water lev...


,site_no,lev_dt,lev_tm,lev_va
14767,473426095052526,2011-02-03,14:00,22.44


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
14767,ds_12,USGS,x473426095052526,x301A,2011-02-03,14:00,22.44,Water level was affected by atmospheric pressure.,Observed,1391.24,...,627.0,21.26,1392.42,424.41,1.18,0.36,1392.249,424.358,changed time from 1201 to match GWSI water lev...,python
13924,ds_12,USGS,473426000000000.0,x301A,2011-02-03,14:00,22.44,static,Other,1391.24,...,627.0,21.26,1392.42,424.41,1.18,0.36,1392.250,424.358,changed time from 1201 to match GWSI water lev...,R


Example of BAD

In [373]:
chk = dfcompare.loc[(dfcompare['LevelDate']=='2011-02-03') & (dfcompare['LocalSiteName']=='x301A')].copy()

In [375]:
chk

,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
25324,ds_12,USGS,x473426095052526,x301A,2011-02-03,14:00,NaN,NaN,Other,NaN,...,627.0,21.26,1392.42,424.41,NaN,NaN,NaN,NaN,changed time from 1201 to match GWSI water lev...,python
14767,ds_12,USGS,x473426095052526,x301A,2011-02-03,15:00,22.44,Water level was affected by atmospheric pressure.,Observed,1391.24,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.24,424.05,NaN,python


In [376]:
chk = dfcompare.loc[(dfcompare['LevelDate']=='2016-01-19') & (dfcompare['LocalSiteName']=='x1428')].copy()
chk

,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
25225,ds_12,USGS,x473419095052402,x1428,2016-01-19,15:00,NaN,NaN,Other,NaN,...,19960.0,6.94,1391.06,423.995,NaN,NaN,NaN,NaN,"Not much oil in well, but probe was covered in...",python
2197,ds_12,USGS,x473419095052402,x1428,2016-01-19,16:00,6.96,Water level was affected by atmospheric pressure.,Observed,1391.04,...,NaN,NaN,NaN,NaN,0.0,0.0,1391.04,423.989,NaN,python


In [ ]:
dfcompare

## 1 HOUR OFF >:()

In [514]:
# Read in R output after investigating the encoding

import chardet
file_path = r"P:\0083\analysis\DataCompilation\DataCompilationR\CreateMasterWaterOilLevelDataSet\outputs\2021-04-27_tblds_12_WaterAndOilLevels.csv"
file_path = r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\EXAMPLE_tblds_12_WaterAndOilLevels.csv"

# (r)ead a file's (b)inary
file = open(file_path, 'rb')
# store as raw data
raw_data = file.read()
# close the file
file.close()
# check encoding with chardet
result = chardet.detect(raw_data)
print("Result: ", result)
encoding = result['encoding']
print(f"Detected potential encoding: {encoding}")

# use the encoding to read the csv
dfr = pd.read_csv(file_path, encoding=encoding)

# if reading in the EXAMPLE file skip the first two rows 
if file_path == r"P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\EXAMPLE_tblds_12_WaterAndOilLevels.csv":
    print('Skipping first two rows for EXAMPLE data')
    dfr = pd.read_csv(file_path, encoding=encoding, skiprows=2)

Result:  {'encoding': 'Windows-1252', 'confidence': 0.73, 'language': ''}
Detected potential encoding: Windows-1252
Skipping first two rows for EXAMPLE data


In [515]:
dfpy = out_MasterWaterOilLevels_DataRelease.copy()

In [516]:
display(dfr.head(2))
display(dfpy.head(2))

,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,WaterLevel_mASL_NAVD88,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel
0,ds_12,USGS,4.733560e+14,xGrant Creek 01,6/28/2019,10:15,0.00,static,Differential Global Positioning System. This c...,1390.17,423.724,NaN,NaN,NaN,NaN,0.0,0.0,1390.17,423.724,NaN
1,ds_12,USGS,4.733580e+14,x426,5/5/1987,12:01,-0.63,static,Steel-tape,1395.35,425.303,NaN,NaN,NaN,NaN,0.0,0.0,1395.35,425.303,NaN


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,WaterLevel_mASL_NAVD88,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel
0,ds_12,USGS,x473356095043701,xGrant Creek 01,2019-06-28,08:56,0.0,static,Differential Global Positioning System. This c...,1390.17,423.724,NaN,NaN,NaN,NaN,0.0,0.0,1390.17,423.724,NaN
1,ds_12,USGS,x473356095043701,xGrant Creek 01,2020-08-06,12:00,-1.6,static,Differential Global Positioning System. This c...,1391.77,424.211,NaN,NaN,NaN,NaN,0.0,0.0,1391.77,424.211,NaN


In [517]:
display(dfr['LevelDate'].head(2))

# Convert date to match python's output
dfr['LevelDate'] = pd.to_datetime(dfr['LevelDate'])
dfr['LevelDate'] = dfr['LevelDate'].dt.strftime('%Y-%m-%d')

display(dfr['LevelDate'].head(2))

0    6/28/2019
1     5/5/1987
Name: LevelDate, dtype: object

0    2019-06-28
1    1987-05-05
Name: LevelDate, dtype: object

In [518]:
# add origin col
dfpy['origin'] = 'python'
dfr['origin'] = 'R'

In [530]:
dfcompare = pd.concat([dfpy, dfr], axis=0)
dfcompare.keys()

Index(['dataset_id', 'AgencyCode', 'USGS_siteno', 'LocalSiteName', 'LevelDate',
       'LevelTime', 'WaterLevel_ftBLS', 'Comments_WaterLevelStatus',
       'Comments_FluidLevelMethod', 'WaterLevel_ftASL_NAVD88',
       'WaterLevel_mASL_NAVD88', 'OilLevelID', 'OilLevel_ftBLS',
       'OilLevel_ftASL_NAVD88', 'OilLevel_mASL_NAVD88', 'OilThickness_ft',
       'OilThickness_m', 'EstWaterLevel_ftASL_NAVD88',
       'EstWaterLevel_mASL_NAVD88', 'Comments_OilLevel', 'origin'],
      dtype='object')

In [532]:
chk = dfcompare.sort_values(by=['LocalSiteName','LevelDate', 'WaterLevel_ftBLS']).copy()
chk = chk[['LocalSiteName', 'LevelDate','LevelTime','WaterLevel_ftBLS', 'OilLevel_ftBLS','OilThickness_ft','Comments_OilLevel','origin']]
chk


,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,OilLevel_ftBLS,OilThickness_ft,Comments_OilLevel,origin
25489,x0501,2005-06-27,12:01,NaN,NaN,NaN,,python
23971,x0501,2005-06-27,12:01,NaN,NaN,NaN,NaN,R
10283,x0501,2012-06-20,13:46,24.00,20.04,3.96,"The oil in this well is trapped, and not scree...",python
9788,x0501,2012-06-20,14:46,24.00,20.04,3.96,"The oil in this well is trapped, and not scree...",R
10284,x0501,2013-07-15,10:38,23.84,19.90,3.94,"Black oil, watery",python
...,...,...,...,...,...,...,...,...
24439,xWL04,2019-06-27,16:34,0.00,NaN,0.00,NaN,python
23111,xWL04,2019-06-27,17:00,0.00,NaN,0.00,NaN,R
24440,xWL04,2020-08-06,12:00,0.31,NaN,0.00,NaN,python
24441,xWL04,2022-08-25,12:16,-0.31,NaN,0.00,NaN,python


0501

In [521]:
# set up vars
my_date = '2012-06-20'
my_site_local = '0501'
my_site_long = '473425095052603'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
19571,19572,0501,Z. Hillman & E. Berquist,2012-06-20,14:46,CDT,22.55,Z,2939,"The oil in this well is trapped, and not scree...",...,NaN,NaN,Z,NaN,NaN,NaN,NaN,NaN,None,"The oil in this well is trapped, and not scree..."


,site_no,lev_dt,lev_tm,lev_va
10283,473425095052603,2012-06-20,13:46,24.0


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
10283,ds_12,USGS,x473425095052603,x0501,2012-06-20,13:46,24.0,Water level was affected by atmospheric pressure.,Calibrated electric tape – accuracy of instrum...,1387.83,...,19572.0,20.04,1391.79,424.218,3.96,1.207,1391.216,424.043,"The oil in this well is trapped, and not scree...",python
9788,ds_12,USGS,473425000000000.0,x0501,2012-06-20,14:46,24.0,static,Calibrated electric tape – accuracy of instrum...,1387.83,...,19572.0,20.04,1391.79,424.218,3.96,1.207,1391.220,424.043,"The oil in this well is trapped, and not scree...",R


1428

In [522]:
# set up vars
my_date = '2015-01-21'
my_site_local = '1428'
my_site_long = '473419095052402'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
19866,19869,1428,A. Berg,2015-01-21,09:30,CST,NaN,Z,2939,No oil,...,NaN,NaN,Z,NaN,NaN,NaN,NaN,NaN,None,No oil;


,site_no,lev_dt,lev_tm,lev_va
2192,473419095052402,2015-01-21,09:30,6.51


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
2192,ds_12,USGS,x473419095052402,x1428,2015-01-21,09:30,6.51,Water level was affected by atmospheric pressure.,Calibrated electric tape – accuracy of instrum...,1391.49,...,19869.0,NaN,NaN,NaN,0.0,0.0,1391.49,424.126,No oil,python
2100,ds_12,USGS,473419000000000.0,x1428,2015-01-21,9:30,6.51,static,Calibrated electric tape – accuracy of instrum...,1391.49,...,19869.0,NaN,NaN,NaN,0.0,0.0,1391.49,424.126,No oil,R


301A

In [523]:
# set up vars
my_date = '2010-12-03'
my_site_local = '301A'
my_site_long = '473426095052526'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
845,625,301A,A. Berg,2010-12-03,12:00,None,21.95,Z,None,,...,NaN,NaN,Z,430.713,423.954,424.024,6.394,6.324,None,None


,site_no,lev_dt,lev_tm,lev_va
14766,473426095052526,2010-12-03,12:00,21.18


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
14766,ds_12,USGS,x473426095052526,x301A,2010-12-03,12:00,21.18,Water level was affected by atmospheric pressure.,Observed,1392.5,...,625.0,20.95,1392.73,424.504,0.23,0.07,1392.697,424.494,,python
13923,ds_12,USGS,473426000000000.0,x301A,2010-12-03,12:00,21.18,static,Other,1392.5,...,625.0,20.95,1392.73,424.504,0.23,0.07,1392.700,424.494,NaN,R


306

In [524]:
# set up vars
my_date = '2011-11-21'
my_site_local = '306'
my_site_long = '473426095052608'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = tblOilLevels8.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
1111,892,306,A. Berg,2011-11-21,13:55,CST,22.3,Z,None,,...,NaN,NaN,Z,430.61,423.218,423.813,6.95,6.355,None,None


,NWTPosition_m,LocalUseCode,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,...,HIF_ID,GWSI_Tape_Correction_ft,OilLevel_ftBMP_corr,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft,OilLevel_ftBLS,lev_meth_cd,Comments_FluidLevelMethod
810,-2.38,wo,892,306,A. Berg,2011-11-21,13:55,CST,22.3,Z,...,NaN,0.0,22.3,473426095052608,1983-05-24,2024-07-22,1.45,20.85,Z,Other


,site_no,lev_dt,lev_tm,lev_va
15863,473426095052608,2011-11-21,13:55,22.8


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
15863,ds_12,USGS,x473426095052608,x306,2011-11-21,13:55,22.8,Water level was affected by atmospheric pressure.,Observed,1390.11,...,892.0,20.85,1392.06,424.3,1.95,0.594,1391.777,424.214,,python
14998,ds_12,USGS,473426000000000.0,x306,2011-11-21,13:55,22.8,static,Other,1390.11,...,892.0,20.85,1392.06,424.3,1.95,0.594,1391.780,424.214,NaN,R


317

In [525]:
# set up vars
my_date = '2016-07-25'
my_site_local = '317'
my_site_long = '473425095052009'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = tblOilLevels8.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
19910,19999,317,A. Berg & A. Witt,2016-07-25,15:34,CDT,31.16,Z,2939,Dark brown oil. Moderate viscosity.,...,NaN,NaN,Z,NaN,NaN,NaN,NaN,NaN,None,Dark brown oil. Moderate viscosity.


,NWTPosition_m,LocalUseCode,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,...,HIF_ID,GWSI_Tape_Correction_ft,OilLevel_ftBMP_corr,GWSI_USGS_siteno,GWSI_MP_BeginDate,GWSI_MP_EndDate,GWSI_MP_height_ft,OilLevel_ftBLS,lev_meth_cd,Comments_FluidLevelMethod
4629,27.55,wo,19999,317,A. Berg & A. Witt,2016-07-25,14:34,CDT,31.16,Z,...,HIF-242570,0.0,31.16,473425095052009,1983-07-02,2024-07-22,1.86,29.3,Z,Other


,site_no,lev_dt,lev_tm,lev_va
7860,473425095052009,2016-07-25,14:34,31.0


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevelID,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin
7860,ds_12,USGS,x473425095052009,x317,2016-07-25,14:34,31.0,Water level was affected by atmospheric pressure.,Observed,1389.47,...,19999.0,29.3,1391.17,424.029,1.7,0.518,1390.924,423.953,Dark brown oil. Moderate viscosity.,python
7487,ds_12,USGS,473425000000000.0,x317,2016-07-25,15:34,31.0,static,Other,1389.47,...,19999.0,29.3,1391.17,424.029,1.7,0.518,1390.920,423.953,Dark brown oil. Moderate viscosity.,R


1217D

In [541]:
# set up vars
my_date = '2015-08-17'
my_site_local = '1217D'
my_site_long = '473429095051005'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel


,site_no,lev_dt,lev_tm,lev_va
21283,473429095051005,2015-08-17,08:31,9.15


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin,pattern
21283,ds_12,USGS,x473429095051005,x1217D,2015-08-17,08:31,9.15,Water level was affected by atmospheric pressure.,Calibrated electric tape – accuracy of instrum...,1387.84,...,NaN,NaN,NaN,0.0,0.0,1387.84,423.014,NaN,python,pythonpythonpython
20102,ds_12,USGS,473429000000000.0,x1217D,2015-08-17,9:31,9.15,static,Calibrated electric tape – accuracy of instrum...,1387.84,...,NaN,NaN,NaN,0.0,0.0,1387.84,423.014,NaN,R,RRR


501C 

In [544]:
# set up vars
my_date = '2004-11-29'
my_site_local = '501C'
my_site_long = '473429095051005'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
#display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel
8335,8057,501C,None,2004-11-29,12:01,None,NaN,O,None,"water level column said ""ice""",...,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,BM,"water level column said ""ice"";"


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin,pattern
19935,ds_12,USGS,473429000000000.0,x501C,2004-11-29,12:01,NaN,Water level was affected by ice.,Observed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,RRR


506

In [550]:
# set up vars
my_date = ('2012-05-07') or ('2012-05-08')
my_date = '2012-05-08'
my_site_local = '506'
my_site_long = '473428095052201'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel


,site_no,lev_dt,lev_tm,lev_va
20310,473428095052201,2012-05-08,18:28,28.75


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin,pattern
20310,ds_12,USGS,x473428095052201,x506,2012-05-08,18:28,28.75,Water level was affected by atmospheric pressure.,Calibrated electric tape – accuracy of instrum...,1390.96,...,NaN,NaN,NaN,0.0,0.0,1390.96,423.965,NaN,python,pythonpythonpython


In [551]:
# set up vars
my_date = ('2012-05-07') or ('2012-05-08')
my_date = '2012-05-07'
my_site_local = '506'
my_site_long = '473428095052201'

chk = tblOilLevels.copy()
chk = chk.loc[(chk['OilLevelDate']==my_date) & (chk['LocalSiteName']== my_site_local)]
display(chk)

chk = dfwl[['site_no', 'lev_dt', 'lev_tm', 'lev_va']].copy()
chk = chk.loc[(chk['site_no']==my_site_long)&(chk['lev_dt']==my_date)]
display(chk)

chk = dfcompare.loc[(dfcompare['LevelDate']==my_date) & (dfcompare['LocalSiteName']==('x'+ my_site_local))].copy()
display(chk)

,OilLevelID,LocalSiteName,PersonMakingMeasurement,OilLevelDate,OilLevelTime,TimeDatum,OilLevel_ftBMP,OilLevelMethod,Oil_Tape_Serial_No,Comments_OilLevel,...,ARCHIVE_Hold_ft,ARCHIVE_Cut_ft,ARCHIVE_NWIS_WaterLevelMethod,ARCHIVE_MP_Elev_meters,ARCHIVE_WaterLevelElevation_meters,ARCHIVE_OilLevelElevation_ft,ARCHIVE_WaterLevelDepth_metersBLS,ARCHIVE_OilLevelDepth_metersBLS,ARCHIVE_BrentComment,ARCHIVE_Comments_OilLevel


,site_no,lev_dt,lev_tm,lev_va


,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin,pattern
19136,ds_12,USGS,473428000000000.0,x506,2012-05-07,19:28,28.75,static,Calibrated electric tape – accuracy of instrum...,1390.96,...,NaN,NaN,NaN,0.0,0.0,1390.96,423.965,NaN,R,RRR


In [533]:
# save the output
dfcompare.sort_values(by=['LocalSiteName','LevelDate', 'WaterLevel_ftBLS']).to_csv(r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\tests\compare_-6.csv')

In [542]:
# check if r every repeats twice
chk = dfcompare.sort_values(by=['LocalSiteName','LevelDate', 'WaterLevel_ftBLS']).copy()
chk['pattern'] = chk['origin'].shift(1) + chk['origin'] + chk['origin'].shift(-1)
chk.to_csv(r'P:\0083\analysis\DataCompilation\DataCompilationPy\create_master_oil_levels\outputs\tests\compare_-6_RR.csv')
chk = chk[chk['pattern'].str.contains('RR', na=False)]

In [543]:
chk

,dataset_id,AgencyCode,USGS_siteno,LocalSiteName,LevelDate,LevelTime,WaterLevel_ftBLS,Comments_WaterLevelStatus,Comments_FluidLevelMethod,WaterLevel_ftASL_NAVD88,...,OilLevel_ftBLS,OilLevel_ftASL_NAVD88,OilLevel_mASL_NAVD88,OilThickness_ft,OilThickness_m,EstWaterLevel_ftASL_NAVD88,EstWaterLevel_mASL_NAVD88,Comments_OilLevel,origin,pattern
20102,ds_12,USGS,473429000000000.0,x1217D,2015-08-17,9:31,9.15,static,Calibrated electric tape – accuracy of instrum...,1387.84,...,NaN,NaN,NaN,0.0,0.0,1387.84,423.014,NaN,R,pythonRR
20103,ds_12,USGS,473429000000000.0,x1217D,2015-10-21,19:00,9.38,static,Calibrated electric tape – accuracy of instrum...,1387.61,...,NaN,NaN,NaN,0.0,0.0,1387.61,422.944,NaN,R,RRpython
14315,ds_12,USGS,473426000000000.0,x303,1983-06-16,12:00,22.08,static,Steel-tape,1391.49,...,NaN,NaN,NaN,0.0,0.0,1391.49,424.126,NaN,R,pythonRR
14316,ds_12,USGS,473426000000000.0,x303,1983-06-16,12:01,22.08,static,Steel-tape,1391.49,...,NaN,NaN,NaN,0.0,0.0,1391.49,424.126,NaN,R,RRpython
14632,ds_12,USGS,473426000000000.0,x303,2012-04-10,10:58,21.98,static,Calibrated electric tape – accuracy of instrum...,1391.59,...,NaN,NaN,NaN,0.0,0.0,1391.59,424.157,NaN,R,pythonRR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9514,ds_12,USGS,473425000000000.0,x9621,2010-07-27,16:23,23.98,static,Calibrated electric tape – accuracy of instrum...,1392.01,...,NaN,NaN,NaN,0.0,0.0,1392.01,424.285,NaN,R,RRpython
9530,ds_12,USGS,473425000000000.0,x9621,2011-12-20,14:46,24.05,static,Calibrated electric tape – accuracy of instrum...,1391.94,...,NaN,NaN,NaN,0.0,0.0,1391.94,424.263,NaN,R,pythonRR
9531,ds_12,USGS,473425000000000.0,x9621,2012-01-24,13:53,NaN,The site was dry (no water level is recorded).,Calibrated electric tape – accuracy of instrum...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R,RRpython
3495,ds_12,USGS,473421000000000.0,x9714,2005-08-22,12:01,9.13,static,Calibrated electric tape – accuracy of instrum...,1395.51,...,NaN,NaN,NaN,0.0,0.0,1395.51,425.351,NaN,R,pythonRR


In [7]:
import os, json, sys, datetime, time, multiprocessing, shutil, subprocess, getopt

In [8]:
import get_mps

get_mps.py 
	-f <AQ folder names; multiples separated by commas ex. Louisiana,Alabama,Mississippi>
	 If folder name contains spaces or wildcards, place it in quotes.


SystemExit: 2